# APAN5310 Group 11 ETL Code
- Schema Creation
- Data Manipulation 
- Data Insertion
- Query Testing

## Create the Schemas for our relational database design
- Using psycopg
- Based on previous sql code
- Database Name: 5310_Group11_ETL_Update

In [438]:
#Connect to PostgreSQL local database
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="5310_Group11_ETL_Update",
    user="postgres",
    password="123")

# create a cursor
cur = conn.cursor()

In [439]:
#Create the Category Table
createcategory = """
CREATE TABLE Category (
	category_id           INT,
	category_name         VARCHAR(30)   NOT NULL,
	PRIMARY KEY (category_id)
);
"""
cur.execute(createcategory)

In [440]:
#Create the Product Table
createproduct = """
CREATE TABLE Product (
	product_id        INT,
	product_name      VARCHAR(30) NOT NULL,
	category_id       INT NOT NULL,
	product_description VARCHAR(300),
	PRIMARY KEY (product_id),
	FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
"""
cur.execute(createproduct)

In [441]:
#Create the ProductPrice Table
createproductprice = """
CREATE TABLE ProductPrice (
	product_id		INT,
    price_date      DATE,
    product_price   NUMERIC(6, 2)   NOT NULL,
    PRIMARY KEY (product_id, price_date),
	FOREIGN KEY (product_id) REFERENCES Product(product_id)
);
"""
cur.execute(createproductprice)

In [442]:
#Create the Supplier Table
createsupplier = """
CREATE TABLE Supplier (
	supplier_id   INT,
	name          VARCHAR(30) NOT NULL,
	email         VARCHAR(50) NOT NULL,
	PRIMARY KEY (supplier_id)
);
"""
cur.execute(createsupplier)

In [443]:
#Create Store Table
createstore = """
CREATE TABLE Store (
	store_id 		INT PRIMARY KEY,
	store_name 		VARCHAR (100) NOT NULL,
	contact_number 	VARCHAR (100) NOT NULL,
	street 			VARCHAR (100) NOT NULL,
    city 			VARCHAR (100) NOT NULL,
    state 			VARCHAR (100) NOT NULL
);
"""
cur.execute(createstore)

In [444]:
#Create LoyaltyProgram Table
createloyaltyprogram = """
CREATE TABLE LoyaltyProgram (
	loyalty_id 		INT PRIMARY KEY,
	level 			VARCHAR (25) NOT NULL,
	points 			INT NOT NULL
);
"""
cur.execute(createloyaltyprogram)

In [445]:
#Create Customer Table
createcustomer = """
CREATE TABLE Customer (
	customer_id 	INT PRIMARY KEY,
	first_name 		VARCHAR (50) NOT NULL,
	last_name 		VARCHAR (50) NOT NULL,
	email 			VARCHAR (200),
	phone 			VARCHAR (10) NOT NULL,
	loyalty_id 		Int NOT NULL,
	FOREIGN KEY (loyalty_id) REFERENCES LoyaltyProgram(loyalty_id)
);
"""
cur.execute(createcustomer)

In [446]:
#Create Employee Table
createemployee = """
CREATE TABLE Employee (
	employee_id 	INT PRIMARY KEY,
	first_name 		VARCHAR (25) NOT NULL,
	last_name 		VARCHAR (25) ,
	position 		VARCHAR (25) NOT NULL,
	store_id 		INT NOT NULL,
	salary 			NUMERIC (10,2) NOT NULL,
	FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""
cur.execute(createemployee)

In [447]:
#Create Orders Table
createorders = """
CREATE TABLE Orders (
	order_id            VARCHAR(200),
	supplier_id         INT NOT NULL,
	stock_date          DATE,
	PRIMARY KEY (order_id),
	FOREIGN KEY (supplier_id) REFERENCES Supplier(supplier_id)
);
"""
cur.execute(createorders)

In [448]:
#Create Sales Table
createsales = """
CREATE TABLE Sales (
	sales_id 		INT PRIMARY KEY,
	customer_id 	INT NOT NULL,
	employee_id 	INT NOT NULL,
	store_id 		INT NOT NULL,
	sales_date 		DATE NOT NULL,
	FOREIGN KEY (customer_id) REFERENCES Customer(customer_id),
	FOREIGN KEY (employee_id) REFERENCES Employee(employee_id),
	FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""
cur.execute(createsales)

In [449]:
#Create stores_orders Table
createstoresorders = """
CREATE TABLE stores_orders (
	order_id		VARCHAR(200),
	store_id		INT,
    PRIMARY KEY (order_id, store_id),
	FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""
cur.execute(createstoresorders)

In [450]:
#Create product_orders Table
createproductorders = """
CREATE TABLE product_orders (
	product_id		INT,
	order_id		VARCHAR(200),
    expiration_date	DATE,
	quantity		INT NOT NULL,
	unit_price		NUMERIC(10,2) NOT NULL,
	PRIMARY KEY (product_id, order_id, expiration_date),
	FOREIGN KEY (product_id) REFERENCES Product(product_id),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);
"""
cur.execute(createproductorders)

In [451]:
#Create SpecificSales Table
createspecificsales = """
CREATE TABLE SpecificSales (
	sales_id            INT,
	product_id          INT,
	quantity		    INT,
    price_paid			NUMERIC(10,2),
	PRIMARY KEY (product_id, sales_id),
	FOREIGN KEY (product_id) REFERENCES Product(product_id),
	FOREIGN KEY (sales_id) REFERENCES Sales(sales_id)
);
"""
cur.execute(createspecificsales)

In [452]:
#Create Payment Table
createpayment = """
CREATE TABLE Payment (
	payment_id 		INT PRIMARY KEY,
	sales_id 		INT NOT NULL,
	payment_method 	VARCHAR (50) NOT NULL,
	discount 		INT,
	tax_amount 		NUMERIC(10,2) NOT NULL,
	total_payment 	NUMERIC(10,2),
	FOREIGN KEY (sales_id) REFERENCES Sales(sales_id)
);
"""
cur.execute(createpayment)

In [453]:
#Create CustomerRating Table
createcustomerrating = """
CREATE TABLE CustomerRating (
	customer_id		INT,
	store_id		INT,
	review_date 	TIMESTAMP,
	rating 			SMALLINT NOT NULL CHECK (rating BETWEEN 1 AND 5),
	review 			TEXT,
	PRIMARY KEY (customer_id, store_id, review_date),
	FOREIGN KEY (customer_id) REFERENCES Customer(customer_id),
	FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""
cur.execute(createcustomerrating)

In [454]:
#Create OperatingCost Table
createoperatingcost = """
CREATE TABLE OperatingCost (
	store_id 		INT,
	date 			DATE,
	fixed_cost 		NUMERIC(15,2),
	variable_cost 	NUMERIC(15,2),
	cost_description	VARCHAR(100) NOT NULL,
	PRIMARY KEY (store_id, date),
	FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""
cur.execute(createoperatingcost)

In [455]:
#Make the changes to the database persistent
conn.commit()

## Data Manipulation for Table Value Insertion
- Create two data frames that includes most of the values that we are going to use in the dable
- Generate additional columns that we have in the SQL database (Such as id, phone number, etc)
- Split the data for each table based on schema attributes

### Data Manipulation Table 1 - Product Side Table
- Merge the CSV file to create a dataset for 8 tables:

- Category Table
- Product Price Table
- Product Table
- product_orders Table
- Supplier Table
- Orders Table
- store_orders Table
- Store Table

This allow us to capture the relationship between each table (such as foreign key constraints) and split the dataset further to each table's attributes

csv files Used for this join dataset:
- product_vendor_orders



In [34]:
#Import Pandas & Numpy for data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import random

In [39]:
#Name the general datdframe df1 so that additiona columns can be created based on the existing columns
df1 = pd.read_csv('products_vendor_orders.csv')

In [40]:
print(df1.head())
print(len(df1))
#There are 5000 rows in this dataframe


                    Vendor Name                Vendor Email  \
0  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
2  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
3  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
4  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   

                           Order Number                    Store Name  \
0  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
1  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
2  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
3  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
4  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   

  Delivery Date Product Name  \
0    2024-06-09        Water   
1    2024-06-09  Cough Syrup   
2    2024-06-09   Beef Steak   
3    2024-06-09       Tulips   
4    2024-06-09      L

#### Add additional attribute for Category Table
- Since the dataset already has Category column
- We need to create a category_id for this Category Table in Database

In [41]:
#Create category_id column based on existing categories
category_labels, unique_categories = pd.factorize(df1['Category'])

#Add the category_id to the dataframe
df1['category_id'] = category_labels + 1

print(df1.head())

category_mapping = {i + 1: category for i, category in enumerate(unique_categories)}
print(category_mapping)


                    Vendor Name                Vendor Email  \
0  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
2  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
3  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
4  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   

                           Order Number                    Store Name  \
0  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
1  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
2  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
3  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
4  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   

  Delivery Date Product Name  \
0    2024-06-09        Water   
1    2024-06-09  Cough Syrup   
2    2024-06-09   Beef Steak   
3    2024-06-09       Tulips   
4    2024-06-09      L

In [46]:
print(df1['Category'].unique())
print(category_mapping)

['beverages' 'pharmacy' 'meats' 'floral' 'fresh produce' 'dairy' 'seafood'
 'frozen foods' 'bakery' 'deli' 'dry goods']
{1: 'beverages', 2: 'pharmacy', 3: 'meats', 4: 'floral', 5: 'fresh produce', 6: 'dairy', 7: 'seafood', 8: 'frozen foods', 9: 'bakery', 10: 'deli', 11: 'dry goods'}


#### Add additional attribute for Product Table
- We need to create a product_id for this Product Table in Database

In [47]:
#Generate product_id based on existing Product column, add 1 to start id at 1
product_labels, unique_products = pd.factorize(df1['Product Name'])
df1['product_id'] = product_labels + 1

print(df1.head())

product_mapping = {i + 1: product for i, product in enumerate(unique_products)}
print(product_mapping)

                    Vendor Name                Vendor Email  \
0  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
2  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
3  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
4  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   

                           Order Number                    Store Name  \
0  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
1  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
2  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
3  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
4  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   

  Delivery Date Product Name  \
0    2024-06-09        Water   
1    2024-06-09  Cough Syrup   
2    2024-06-09   Beef Steak   
3    2024-06-09       Tulips   
4    2024-06-09      L

In [49]:
print(df1['Product Name'].unique())
print(product_mapping)
#There are 33 unique products in total

['Water' 'Cough Syrup' 'Beef Steak' 'Tulips' 'Lettuce' 'Cheese'
 'Band-Aids' 'Shrimp' 'Carrots' 'Frozen Vegetables' 'Chicken Breast'
 'Juice' 'Bread' 'Roast Beef' 'Pork Loin' 'Yogurt' 'Ham' 'Lobster'
 'Salmon Fillet' 'Frozen Pizza' 'Sunflowers' 'Bagel' 'Soda' 'Beans'
 'Roses' 'Croissant' 'Ice Cream' 'Apples' 'Turkey Breast' 'Milk' 'Pasta'
 'Rice' 'Aspirin']
{1: 'Water', 2: 'Cough Syrup', 3: 'Beef Steak', 4: 'Tulips', 5: 'Lettuce', 6: 'Cheese', 7: 'Band-Aids', 8: 'Shrimp', 9: 'Carrots', 10: 'Frozen Vegetables', 11: 'Chicken Breast', 12: 'Juice', 13: 'Bread', 14: 'Roast Beef', 15: 'Pork Loin', 16: 'Yogurt', 17: 'Ham', 18: 'Lobster', 19: 'Salmon Fillet', 20: 'Frozen Pizza', 21: 'Sunflowers', 22: 'Bagel', 23: 'Soda', 24: 'Beans', 25: 'Roses', 26: 'Croissant', 27: 'Ice Cream', 28: 'Apples', 29: 'Turkey Breast', 30: 'Milk', 31: 'Pasta', 32: 'Rice', 33: 'Aspirin'}


#### Add additional attribute for ProductPrice Table
- Although there are Price per Unit in the dataframe and the Delivery Date
- We need to create product_price (sales price) based on Price per Unite
- We need to create price_date based on Deliver Date

In [50]:
# Create product_price using a 30% markup on the current order price
df1['product_price'] = df1['Price per Unit'] * 1.30

#Create price_date using Delivery Date by adding 1 day
# Convert 'Delivery Date' to datetime
df1['Delivery Date'] = pd.to_datetime(df1['Delivery Date'])

from datetime import timedelta

df1['price_date'] = df1['Delivery Date'] + timedelta(days=1)

print(df1.head())

                    Vendor Name                Vendor Email  \
0  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
2  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
3  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
4  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   

                           Order Number                    Store Name  \
0  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
1  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
2  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
3  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
4  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   

  Delivery Date Product Name  \
0    2024-06-09        Water   
1    2024-06-09  Cough Syrup   
2    2024-06-09   Beef Steak   
3    2024-06-09       Tulips   
4    2024-06-09      L

#### Add additional attribute for product_orders Table
- product_id is created already
- quantity is included already
- Expiration Date is included already
- unit_price is included already (Price per Unit)
- order_id is included (Order Number)

#### Add additional attribute for Supplier Table
- supplier name exist already (Vendor Name)
- email exist already (Vendor Email)
- Need supplier_id

In [51]:
supplier_labels, unique_suppliers = pd.factorize(df1['Vendor Name'])
df1['supplier_id'] = supplier_labels + 1

print(df1.head())

supplier_mapping = {i + 1: vendor for i, vendor in enumerate(unique_suppliers)}
print(supplier_mapping)

                    Vendor Name                Vendor Email  \
0  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
2  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
3  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   
4  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com   

                           Order Number                    Store Name  \
0  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
1  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
2  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
3  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   
4  140ead02-1500-4660-897e-8773e7c34a6f  ABC Foodmart - Staten Island   

  Delivery Date Product Name  \
0    2024-06-09        Water   
1    2024-06-09  Cough Syrup   
2    2024-06-09   Beef Steak   
3    2024-06-09       Tulips   
4    2024-06-09      L

In [52]:
df1['Vendor Name'].nunique()

50

#### Add additional attribute for Orders Table
- order id already exist (Order Number)
- supplier_id already exist (from previous part)
- stock_date already exist (Delivery Date)

#### Add additional attribute for Store Table
- Load the stores.csv file
- Already have store name
- Already have address (through combining the street, city, and state)
- Need store_id
- Need contact_number

In [55]:
store_insert = pd.read_csv('stores.csv')
print(store_insert.head())
print(len(store_insert))
#There are 400 rows of data

             store_name    store_street store_city store_state  store_zipcode  \
0  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
1  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
2  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
3  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
4  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   

   store_manager store_open_date  store_parking_capacity store_expense_date  \
0  Hoffman, Tina      2017-06-18                      50         2024-04-01   
1  Hoffman, Tina      2017-06-18                      50         2024-03-01   
2  Hoffman, Tina      2017-06-18                      50         2024-02-01   
3  Hoffman, Tina      2017-06-18                      50         2024-01-01   
4  Hoffman, Tina      2017-06-18                      50         2023-12-01   

  store_expense_type  store_expense_am

In [56]:
# Generate store_id 
store_labels, unique_stores = pd.factorize(store_insert['store_name'])
store_insert['store_id'] = store_labels + 1

print(store_insert.head())

store_mapping = {i + 1: store for i, store in enumerate(unique_stores)}
print(store_mapping)

             store_name    store_street store_city store_state  store_zipcode  \
0  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
1  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
2  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
3  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
4  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   

   store_manager store_open_date  store_parking_capacity store_expense_date  \
0  Hoffman, Tina      2017-06-18                      50         2024-04-01   
1  Hoffman, Tina      2017-06-18                      50         2024-03-01   
2  Hoffman, Tina      2017-06-18                      50         2024-02-01   
3  Hoffman, Tina      2017-06-18                      50         2024-01-01   
4  Hoffman, Tina      2017-06-18                      50         2023-12-01   

  store_expense_type  store_expense_am

In [60]:
print(store_insert['store_name'].unique())
print(store_insert['store_name'].nunique())
print(store_insert.isnull().any())

['ABC Foodmart - DUMBO' 'ABC Foodmart - Tribeca'
 'ABC Foodmart - Bay Ridge' 'ABC Foodmart - Whitestone'
 'ABC Foodmart - Staten Island']
5
store_name                   False
store_street                 False
store_city                   False
store_state                  False
store_zipcode                False
store_manager                False
store_open_date              False
store_parking_capacity       False
store_expense_date           False
store_expense_type           False
store_expense_amount         False
store_expense_description    False
store_id                     False
dtype: bool


In [62]:
#Create unique phone numbers for unique stores
store_names = ['ABC Foodmart - DUMBO', 'ABC Foodmart - Tribeca', 'ABC Foodmart - Bay Ridge', 
               'ABC Foodmart - Whitestone', 'ABC Foodmart - Staten Island']

# Corresponding phone numbers for each store
phone_numbers = ['3471231234', '3479879876', '3474564565', '3478768765', '3471234567']

# Create a dictionary mapping store names to phone numbers
phone_mapping = dict(zip(store_names, phone_numbers))

# Map phone numbers to store names in the DataFrame
store_insert['contact_number'] = store_insert['store_name'].map(phone_mapping)

print(store_insert[['store_name', 'contact_number']].head())


             store_name contact_number
0  ABC Foodmart - DUMBO     3471231234
1  ABC Foodmart - DUMBO     3471231234
2  ABC Foodmart - DUMBO     3471231234
3  ABC Foodmart - DUMBO     3471231234
4  ABC Foodmart - DUMBO     3471231234


In [63]:
store_number_check = store_insert.groupby('store_name')['contact_number'].nunique()
print(store_number_check)

store_name
ABC Foodmart - Bay Ridge        1
ABC Foodmart - DUMBO            1
ABC Foodmart - Staten Island    1
ABC Foodmart - Tribeca          1
ABC Foodmart - Whitestone       1
Name: contact_number, dtype: int64


In [64]:
#Map the store_id in the store_insert dataframe to the Store Name in the df1 dataframe
store_id_map = dict(zip(store_insert['store_name'], store_insert['store_id']))

df1['store_id'] = df1['Store Name'].map(store_id_map)

print(df1[['Store Name', 'store_id']].head())

                     Store Name  store_id
0  ABC Foodmart - Staten Island         5
1  ABC Foodmart - Staten Island         5
2  ABC Foodmart - Staten Island         5
3  ABC Foodmart - Staten Island         5
4  ABC Foodmart - Staten Island         5


In [67]:
#Check id match result
merged_df = pd.merge(df1, store_insert, left_on='Store Name', right_on='store_name', suffixes=('_df1', '_store_insert'))

# Add a column to check if the store_ids are the same
merged_df['id_match'] = merged_df['store_id_df1'] == merged_df['store_id_store_insert']

print(merged_df[['Store Name', 'store_id_df1', 'store_name', 'store_id_store_insert', 'id_match']].head())
print("match result:", merged_df['id_match'].all())

                     Store Name  store_id_df1                    store_name  \
0  ABC Foodmart - Staten Island             5  ABC Foodmart - Staten Island   
1  ABC Foodmart - Staten Island             5  ABC Foodmart - Staten Island   
2  ABC Foodmart - Staten Island             5  ABC Foodmart - Staten Island   
3  ABC Foodmart - Staten Island             5  ABC Foodmart - Staten Island   
4  ABC Foodmart - Staten Island             5  ABC Foodmart - Staten Island   

   store_id_store_insert  id_match  
0                      5      True  
1                      5      True  
2                      5      True  
3                      5      True  
4                      5      True  
match result: True


#### Add additional attribute for product_orders Table
- order_id already exist (Order Number)
- store_id already exist


In [70]:
print(df1.columns)

Index(['Vendor Name', 'Vendor Email', 'Order Number', 'Store Name',
       'Delivery Date', 'Product Name', 'Product Description', 'Category',
       'Unit of Measurement', 'Quantity', 'Price per Unit', 'Expiration Date',
       'category_id', 'product_id', 'product_price', 'price_date',
       'supplier_id', 'store_id'],
      dtype='object')


### Generate the dataframe and csv file for these 8 table insertion
- Category Table -- category_df
- Product Price Table -- final_product_price_df
- Product Table -- product_table_df
- product_orders Table
- Supplier Table -- supplier_table_df
- Orders Table -- orders_table_df
- store_orders Table -- stores_orders_table_df
- Store Table -- store_table_df

In [72]:
#Create Category Table
category_df = df1[['category_id', 'Category']].drop_duplicates().rename(columns={'Category': 'category_name'})

print(category_df.head())

#check the number of categories
print(len(category_df))

#Output to csv file
category_df.to_csv('Category Table Insertion.csv', index=False)

   category_id  category_name
0            1      beverages
1            2       pharmacy
2            3          meats
3            4         floral
4            5  fresh produce
11


In [271]:
#Product Price Table
product_price = df1[['product_id', 'price_date', 'product_price']]
print(len(product_price))

5000


In [272]:
#Check whether the combination of product_id and price_date is unique
duplicates = product_price.duplicated(subset=['product_id', 'price_date'], keep=False)
duplicate_count = duplicates.sum()
print(duplicate_count)
print(product_price[duplicates])


2025
      product_id price_date  product_price
10            10 2024-06-23         65.910
14            10 2024-06-23         90.051
16            15 2025-02-14         27.027
22            12 2025-02-26        107.861
24            20 2025-02-26        112.645
...          ...        ...            ...
4986          26 2024-10-11         45.552
4989          17 2024-10-11        108.264
4991           4 2024-07-12          8.190
4995          10 2024-08-28        125.411
4999           7 2024-08-28         77.805

[2025 rows x 3 columns]


In [273]:
#We assume that three stores need to keep a uniform product price since they are all in NY state
#Therefore, we need to check whether the price difference of same product from same vendor are more than 20 dollars

product_price_df = product_price
# Filter rows with duplicate product_id and price_date combinations
duplicates = product_price_df[product_price_df.duplicated(subset=['product_id', 'price_date'], keep=False)]

# Group by product_id and price_date to check price differences
grouped = duplicates.groupby(['product_id', 'price_date'])
price_diff = grouped['product_price'].agg(
    max_price='max', 
    min_price='min', 
    price_difference=lambda x: max(x) - min(x) 
)


significant_diff = price_diff[price_diff['price_difference'] > 20]


print(significant_diff)


print(len(significant_diff))


                       max_price  min_price  price_difference
product_id price_date                                        
1          2024-04-21    116.051     51.987            64.064
           2024-05-21     37.856      4.875            32.981
           2024-05-24    129.701     14.105           115.596
           2024-06-23     75.582     31.928            43.654
           2024-07-08    114.556     61.347            53.209
...                          ...        ...               ...
33         2024-12-21    109.525     24.609            84.916
           2025-01-06    114.192     21.567            92.625
           2025-01-11     83.148     61.334            21.814
           2025-02-05     71.149     45.162            25.987
           2025-03-18     95.251     49.998            45.253

[692 rows x 3 columns]
692


In [274]:
#For price difference that are greater than 20 dollar, keep the rows with higher price
#But adjust the price to 75% of higher price
#For price difference that are below 20 dollar, keep the rows with higher price as is

#Filter rows with duplicate product_id and price_date combinations
duplicates = product_price_df[product_price_df.duplicated(subset=['product_id', 'price_date'], keep=False)]

#Group by product_id and price_date to check price differences
grouped = duplicates.groupby(['product_id', 'price_date'])
price_diff = grouped['product_price'].agg(
    max_price='max',
    min_price='min',
    price_difference=lambda x: max(x) - min(x)
)

#Calculate the adjusted price based on the conditions
def calculate_adjusted_price(row):
    if row['price_difference'] > 20:
        return 0.75 * row['max_price']
    else:
        return row['max_price']

#get the new price
price_diff['adjusted_price'] = price_diff.apply(calculate_adjusted_price, axis=1)

#map the product_id and price_date to the adjusted price
price_mapping = price_diff.reset_index().set_index(['product_id', 'price_date'])['adjusted_price'].to_dict()

#Update to product_price_df
def apply_price_adjustment(row):
    return price_mapping.get((row['product_id'], row['price_date']), row['product_price'])

product_price_df['adjusted_product_price'] = product_price_df.apply(apply_price_adjustment, axis=1)

print(product_price_df.head())

   product_id price_date  product_price  adjusted_product_price
0           1 2024-06-10          3.952                   3.952
1           2 2024-06-10         54.795                  54.795
2           3 2024-06-10         94.224                  94.224
3           4 2024-06-10        107.081                 107.081
4           5 2024-06-10        111.826                 111.826


In [275]:
len(product_price_df)

5000

In [276]:
print(product_price_df.head())

   product_id price_date  product_price  adjusted_product_price
0           1 2024-06-10          3.952                   3.952
1           2 2024-06-10         54.795                  54.795
2           3 2024-06-10         94.224                  94.224
3           4 2024-06-10        107.081                 107.081
4           5 2024-06-10        111.826                 111.826


In [277]:
# Sort by product_id, price_date, and product_price in descending order
product_price_df.sort_values(by=['product_id', 'price_date', 'product_price'], ascending=[True, True, False], inplace=True)

# Remove duplicates by keeping only the row with the highest product_price for each product_id and price_date combination
final_product_price_df = product_price_df.drop_duplicates(subset=['product_id', 'price_date'], keep='first')

# Reset the index
final_product_price_df.reset_index(drop=True, inplace=True)

# Print the result to verify
print(len(final_product_price_df))
print(final_product_price_df.head())

3895
   product_id price_date  product_price  adjusted_product_price
0           1 2024-04-20         37.141                37.14100
1           1 2024-04-21        116.051                87.03825
2           1 2024-04-26        113.932               113.93200
3           1 2024-05-04         46.696                46.69600
4           1 2024-05-06         78.741                78.74100


In [280]:
print(final_product_price_df.tail(40))

      product_id price_date  product_price  adjusted_product_price
3855          33 2024-12-07         56.069                42.05175
3856          33 2024-12-11         97.162                97.16200
3857          33 2024-12-12         64.883                64.88300
3858          33 2024-12-14          6.227                 6.22700
3859          33 2024-12-18         71.448                53.58600
3860          33 2024-12-21        109.525                82.14375
3861          33 2024-12-24         38.974                38.97400
3862          33 2024-12-28          0.767                 0.76700
3863          33 2024-12-31         30.615                30.61500
3864          33 2025-01-01        107.315               107.31500
3865          33 2025-01-04         45.630                45.63000
3866          33 2025-01-06        114.192                85.64400
3867          33 2025-01-07         62.569                62.56900
3868          33 2025-01-08          4.017                 4.0

In [281]:
duplicates_exist = final_product_price_df.duplicated(subset=['product_id', 'price_date'], keep=False).any()
print(duplicates_exist)

False


In [282]:
#Check whether all price_date exist in the product_price dataframe
#Extract sets of price_date values from both dfs
original_dates = set(product_price['price_date'])
final_dates = set(final_product_price_df['price_date'])

#Check if all dates in final df exist in the original df
missing_dates = final_dates - original_dates 
extra_dates = original_dates - final_dates 

# Print the results
if not missing_dates and not extra_dates:
    print("All price_date values exist in the original product_price dataframe.")
else:
    if missing_dates:
        print("These dates arenot in the product_price:", missing_dates)
    if extra_dates:
        print("These dates are in the produce_price but not in final_product_price_df:", extra_dates)

All price_date values exist in the original product_price dataframe.


In [283]:
#Check if all product_id exist in original dataframe
#Extract sets of product_id values from both dfs
original_product_ids = set(product_price['product_id'])
final_product_ids = set(final_product_price_df['product_id'])

#Check if all product_ids in the original df exist in the final df
missing_product_ids = original_product_ids - final_product_ids

if not missing_product_ids:
    print("All product_id values exist in the final df")
else:
    print("These product_id values are missing:", missing_product_ids)

All product_id values exist in the final df


In [323]:
#Drop the product_price column and rename the adjuster_price to product_price
final_product_price_df.drop(columns=['product_price'], inplace=True)

final_product_price_df.rename(columns={'adjusted_product_price': 'product_price'}, inplace=True)


In [324]:
print(final_product_price_df.head())

#Output to csv file
final_product_price_df.to_csv('ProductPrice Table Insertion.csv', index=False)

   product_id price_date  product_price
0           1 2024-04-20       37.14100
1           1 2024-04-21       87.03825
2           1 2024-04-26      113.93200
3           1 2024-05-04       46.69600
4           1 2024-05-06       78.74100


In [96]:
#Create Product Table
product_df = df1.drop_duplicates(subset=['product_id'])
product_df.rename(columns={'Product Name': 'product_name', 'Product Description': 'product_description'}, inplace=True)

product_table_df = product_df[['product_id', 'product_name', 'category_id', 'product_description']]

product_table_df.reset_index(drop=True, inplace=True)

print(product_table_df)

    product_id       product_name  category_id  \
0            1              Water            1   
1            2        Cough Syrup            2   
2            3         Beef Steak            3   
3            4             Tulips            4   
4            5            Lettuce            5   
5            6             Cheese            6   
6            7          Band-Aids            2   
7            8             Shrimp            7   
8            9            Carrots            5   
9           10  Frozen Vegetables            8   
10          11     Chicken Breast            3   
11          12              Juice            1   
12          13              Bread            9   
13          14         Roast Beef           10   
14          15          Pork Loin            3   
15          16             Yogurt            6   
16          17                Ham           10   
17          18            Lobster            7   
18          19      Salmon Fillet            7   


C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\2975485408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df.rename(columns={'Product Name': 'product_name', 'Product Description': 'product_description'}, inplace=True)


In [98]:
#outut to csv file
print(len(product_table_df))
product_table_df.to_csv('Product Table Insertion.csv', index=False)

33


In [101]:
#Create Supplier Table
#Select rows with unique supplier_id
unique_suppliers_df = df1.drop_duplicates(subset=['supplier_id'])

# ename columns to match the Supplier Table schema
unique_suppliers_df.rename(columns={'Vendor Name': 'name', 'Vendor Email': 'email'}, inplace=True)

#Select only the columns needed for the Supplier Table
supplier_table_df = unique_suppliers_df[['supplier_id', 'name', 'email']]

supplier_table_df.reset_index(drop=True, inplace=True)

print(supplier_table_df)

    supplier_id                            name  \
0             1    Wong, Mcdonald and Rodriguez   
1             2                 Hamilton-Burton   
2             3                    Cardenas Ltd   
3             4                    Matthews LLC   
4             5     Hamilton, Briggs and Nguyen   
5             6                   Lowery-Abbott   
6             7                       Silva PLC   
7             8              Patterson and Sons   
8             9       Wallace, Brown and Garcia   
9            10        West, Jones and Hatfield   
10           11                    Giles-Greene   
11           12   Crawford, Oliver and Anderson   
12           13       Garcia, David and Mendoza   
13           14                      Neal-Jones   
14           15                   Molina-Rivera   
15           16                   Gonzalez-Diaz   
16           17          Gray, Roman and Little   
17           18                     Watts-Brown   
18           19                

C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\3393374167.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_suppliers_df.rename(columns={'Vendor Name': 'name', 'Vendor Email': 'email'}, inplace=True)


In [103]:
print(len(supplier_table_df))
supplier_table_df.to_csv('Supplier Table Insertion.csv', index=False)


50


In [107]:
#Create Orders Table
#Select rows with unique order_id
unique_orders_df = df1.drop_duplicates(subset=['Order Number'])

#Rename columns to match the Orders Table schema
unique_orders_df.rename(columns={'Order Number': 'order_id', 'Delivery Date': 'stock_date'}, inplace=True)

#Select only the columns needed for the Orders Table
orders_table_df = unique_orders_df[['order_id', 'supplier_id', 'stock_date']]

orders_table_df.reset_index(drop=True, inplace=True)

print(orders_table_df)

                                 order_id  supplier_id stock_date
0    140ead02-1500-4660-897e-8773e7c34a6f            1 2024-06-09
1    849766ff-6b13-4de9-8169-e3cfb6832bbe            2 2025-01-09
2    4f83ccd2-5dad-4420-89c2-8365a60846d4            3 2024-06-22
3    1fcd017d-8cdb-4901-b874-39e43b3bbb0b            4 2025-02-13
4    a088b206-5765-4a0d-b384-1ed9dc274246            5 2025-02-25
..                                    ...          ...        ...
995  f0950284-d619-4058-a949-17cc5c48988d           39 2024-06-18
996  8e70f7c2-f2ee-4eb9-9c34-6b0482af8b56           33 2025-03-07
997  4d9416ba-8a88-4576-ac70-ad38b25a9ca5           45 2024-10-10
998  cd94409b-7069-4d8b-b71d-55ddd0bacb93           17 2024-07-11
999  5f70ecf0-a595-482e-987b-9b59bec1695e           28 2024-08-27

[1000 rows x 3 columns]


C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\3694726418.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_orders_df.rename(columns={'Order Number': 'order_id', 'Delivery Date': 'stock_date'}, inplace=True)


In [109]:
print(len(orders_table_df))
orders_table_df.to_csv('Orders Table Insertion.csv', index=False)

1000


In [112]:
#Create Store Table
#Select rows with unique store_id
unique_stores_df = store_insert.drop_duplicates(subset=['store_id'])

#Select only the columns needed for the Store Table and rename them accordingly
store_table_df = unique_stores_df[['store_id', 'store_name', 'contact_number', 'store_street', 'store_city', 'store_state']]
store_table_df.rename(columns={'store_street': 'street', 'store_city': 'city', 'store_state': 'state'}, inplace=True)

store_table_df.reset_index(drop=True, inplace=True)

print(store_table_df)

   store_id                    store_name contact_number  \
0         1          ABC Foodmart - DUMBO     3471231234   
1         2        ABC Foodmart - Tribeca     3479879876   
2         3      ABC Foodmart - Bay Ridge     3474564565   
3         4     ABC Foodmart - Whitestone     3478768765   
4         5  ABC Foodmart - Staten Island     3471234567   

                     street           city state  
0            123 Jay Street       Brooklyn    NY  
1       133 Chambers Street       New York    NY  
2              8518 3rd Ave       Brooklyn    NY  
3  153-65 Cross Island Pkwy     Whitestone    NY  
4         2655 Richmond Ave  Staten Island    NY  


C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\922714924.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_table_df.rename(columns={'store_street': 'street', 'store_city': 'city', 'store_state': 'state'}, inplace=True)


In [114]:
print(len(store_table_df))
store_table_df.to_csv('Store Table Insertion.csv', index=False)

5


In [117]:
#Create stores_orders Table
#Select rows with unique combination of store_id and order_id
unique_stores_orders_df = df1.drop_duplicates(subset=['store_id', 'Order Number'])

#Rename the order_id column to match the schema
unique_stores_orders_df.rename(columns={'Order Number': 'order_id'}, inplace=True)

# Select only the columns needed for the Stores_Orders Table
stores_orders_table_df = unique_stores_orders_df[['store_id', 'order_id']]

stores_orders_table_df.reset_index(drop=True, inplace=True)

print(stores_orders_table_df)

     store_id                              order_id
0           5  140ead02-1500-4660-897e-8773e7c34a6f
1           5  849766ff-6b13-4de9-8169-e3cfb6832bbe
2           3  4f83ccd2-5dad-4420-89c2-8365a60846d4
3           3  1fcd017d-8cdb-4901-b874-39e43b3bbb0b
4           3  a088b206-5765-4a0d-b384-1ed9dc274246
..        ...                                   ...
995         2  f0950284-d619-4058-a949-17cc5c48988d
996         2  8e70f7c2-f2ee-4eb9-9c34-6b0482af8b56
997         3  4d9416ba-8a88-4576-ac70-ad38b25a9ca5
998         5  cd94409b-7069-4d8b-b71d-55ddd0bacb93
999         1  5f70ecf0-a595-482e-987b-9b59bec1695e

[1000 rows x 2 columns]


C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\3505515514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_stores_orders_df.rename(columns={'Order Number': 'order_id'}, inplace=True)


In [118]:
print(len(stores_orders_table_df))
stores_orders_table_df.to_csv('storesorders Table Insertion.csv', index=False)

1000


In [121]:
#Create product_orders Table
#Select rows with unique combination of product_id, Order Number, and expiration_date
unique_product_orders_df = df1.drop_duplicates(subset=['product_id', 'Order Number', 'Expiration Date'])

#Rename columns to match the Product_Orders Table schema
unique_product_orders_df.rename(columns={'Order Number': 'order_id', 'Quantity': 'quantity', 'Expiration Date': 'expiration_date', 'Price per Unit': 'unit_price'}, inplace=True)

#Select only the columns needed for the Product_Orders Table
product_orders_table_df = unique_product_orders_df[['product_id', 'order_id', 'quantity', 'expiration_date', 'unit_price']]

product_orders_table_df.reset_index(drop=True, inplace=True)

print(product_orders_table_df)


      product_id                              order_id  quantity  \
0              1  140ead02-1500-4660-897e-8773e7c34a6f        18   
1              2  140ead02-1500-4660-897e-8773e7c34a6f       100   
2              3  140ead02-1500-4660-897e-8773e7c34a6f        93   
3              4  140ead02-1500-4660-897e-8773e7c34a6f        94   
4              5  140ead02-1500-4660-897e-8773e7c34a6f        76   
...          ...                                   ...       ...   
4994          10  5f70ecf0-a595-482e-987b-9b59bec1695e        16   
4995          27  5f70ecf0-a595-482e-987b-9b59bec1695e        74   
4996          17  5f70ecf0-a595-482e-987b-9b59bec1695e        41   
4997           4  5f70ecf0-a595-482e-987b-9b59bec1695e        53   
4998           7  5f70ecf0-a595-482e-987b-9b59bec1695e        42   

     expiration_date  unit_price  
0         2024-08-21        3.04  
1         2025-02-27       42.15  
2         2024-11-24       72.48  
3         2025-05-22       82.37  
4       

C:\Users\Michael\AppData\Local\Temp\ipykernel_34492\389496945.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_product_orders_df.rename(columns={'Order Number': 'order_id', 'Quantity': 'quantity', 'Expiration Date': 'expiration_date', 'Price per Unit': 'unit_price'}, inplace=True)


In [124]:
print(len(product_orders_table_df))
product_orders_table_df.to_csv('productorders Table Insertion.csv', index=False)

4999


### Data Manipulation Table 2 - Sales Side Table
- Merge the CSV file to create a dataset for 7 tables:

- Sales Table
- Specific Sales Table
- Payment Table
- Customer Table
- Loyalty Program Table
- Employee Table
- Customer Rating Table

This allow us to capture the relationship between each table (such as foreign key constraints) and split the dataset further to each table's attributes

csv files Used for this join dataset:
- customer_sales
- employee_shifts

Need to use the ids from previous tables for this column generation

#### Create Employee Table From Scratch before joining tables
- Since Employee works in Sales table and is related to Store table
- It is important to generate the dataframe for the Employee Table first
- Use the employees_shifts.csv for data manipulation
- Use Stores Table to map the store_id to the Employee dataframe

In [126]:
#import data and inspect
employee_df = pd.read_csv('employees_shifts.csv')
print(employee_df.head())
print(len(employee_df))
#There are 5000 rows in total

      Employee Name      Manager Name                            Email  \
0         Sean, Mr.  Compton, Jeffrey         mr..sean@abcfoodmart.com   
1  Foster, Jennifer     Hoffman, Tina  jennifer.foster@abcfoodmart.com   
2      Shah, Amanda      Ellis, Grant      amanda.shah@abcfoodmart.com   
3     Garcia, Sarah    Cooper, Robert     sarah.garcia@abcfoodmart.com   
4     Riley, Ashley     Hoffman, Tina     ashley.riley@abcfoodmart.com   

  Date of Hire                         Store       Shift Start  \
0   2023-03-25  ABC Foodmart - Staten Island  2023-04-14 00:00   
1   2023-06-15          ABC Foodmart - DUMBO  2023-07-08 00:00   
2   2023-10-07          ABC Foodmart - DUMBO  2023-10-18 00:00   
3   2022-12-14        ABC Foodmart - Tribeca  2023-01-06 00:00   
4   2022-08-17          ABC Foodmart - DUMBO  2022-08-27 00:00   

          Shift End           Station  Hourly Pay Rate  
0  2023-04-14 00:00           seafood            12.60  
1  2023-07-08 00:00         dry goods       

In [127]:
#Map the store_id to the employee dataframe first
#Create a dictionary mapping store names to store IDs
store_mapping = store_table_df.set_index('store_name')['store_id'].to_dict()

#Map store_name to store_id in the Employee DataFrame
employee_df['store_id'] = employee_df['Store'].map(store_mapping)

# Print the updated Employee DataFrame with store_id column
print(employee_df)

         Employee Name          Manager Name                            Email  \
0            Sean, Mr.      Compton, Jeffrey         mr..sean@abcfoodmart.com   
1     Foster, Jennifer         Hoffman, Tina  jennifer.foster@abcfoodmart.com   
2         Shah, Amanda          Ellis, Grant      amanda.shah@abcfoodmart.com   
3        Garcia, Sarah        Cooper, Robert     sarah.garcia@abcfoodmart.com   
4        Riley, Ashley         Hoffman, Tina     ashley.riley@abcfoodmart.com   
...                ...                   ...                              ...   
4995  Kramer, Veronica  Farrell, Christopher  veronica.kramer@abcfoodmart.com   
4996    Golden, Laurie          Waller, Chad    laurie.golden@abcfoodmart.com   
4997     Gross, Andrew     Johnston, William     andrew.gross@abcfoodmart.com   
4998  Dodson, Courtney           Moran, John  courtney.dodson@abcfoodmart.com   
4999  Blanchard, Ricky        Flores, Angela  ricky.blanchard@abcfoodmart.com   

     Date of Hire          

In [128]:
#Varify whether the mapping relationship is the same across two tables
#Create a set of unique store_id from the store_table_df
store_ids_store_table = set(store_table_df['store_id'])

#Create a set of unique store_id from the employee_df
store_ids_employee = set(employee_df['store_id'])

#Check if the sets of unique store IDs are equal
if store_ids_store_table == store_ids_employee:
    print("The mapping is the same for both tables.")
else:
    print("The mapping is different for both tables.")

The mapping is the same for both tables.


In [129]:
#In our observation, we saw that the format of the names are different
df2_with_prefixes = employee_df[employee_df['Employee Name'].str.contains(r',\s*Mr\.|Ms\.|Dr\.|Mrs\.', regex=True)]

In [130]:
len(df2_with_prefixes)

251

In [131]:
import warnings
warnings.filterwarnings('ignore')
df2_with_prefixes[['first_name', 'last_name']] = df2_with_prefixes['Employee Name'].apply(
    lambda x: pd.Series(x.split(', ')) if x.count(',') == 1 else pd.Series([None, None])
)

In [132]:
print(len(df2_with_prefixes))
print(df2_with_prefixes.head())

251
   Employee Name      Manager Name                       Email Date of Hire  \
0      Sean, Mr.  Compton, Jeffrey    mr..sean@abcfoodmart.com   2023-03-25   
49   Terri, Mrs.       Moran, John  mrs..terri@abcfoodmart.com   2022-09-23   
50     Eric, Mr.      Waller, Chad    mr..eric@abcfoodmart.com   2022-09-02   
64   Terri, Mrs.        David, Mr.  mrs..terri@abcfoodmart.com   2022-09-23   
72    Bryan, Mr.     Hoffman, Tina   mr..bryan@abcfoodmart.com   2023-08-26   

                           Store       Shift Start         Shift End  \
0   ABC Foodmart - Staten Island  2023-04-14 00:00  2023-04-14 00:00   
49     ABC Foodmart - Whitestone  2022-10-04 00:00  2022-10-04 00:00   
50        ABC Foodmart - Tribeca  2022-09-22 00:00  2022-09-22 00:00   
64        ABC Foodmart - Tribeca  2022-10-28 00:00  2022-10-28 00:00   
72  ABC Foodmart - Staten Island  2023-08-28 00:00  2023-08-28 00:00   

          Station  Hourly Pay Rate  store_id first_name last_name  
0         seafood   

In [133]:
#Since these 251 employees do not have last name shown in original dataset, we give this a None value 
df2_with_prefixes = df2_with_prefixes.assign(last_name=None)
print(df2_with_prefixes.head())

   Employee Name      Manager Name                       Email Date of Hire  \
0      Sean, Mr.  Compton, Jeffrey    mr..sean@abcfoodmart.com   2023-03-25   
49   Terri, Mrs.       Moran, John  mrs..terri@abcfoodmart.com   2022-09-23   
50     Eric, Mr.      Waller, Chad    mr..eric@abcfoodmart.com   2022-09-02   
64   Terri, Mrs.        David, Mr.  mrs..terri@abcfoodmart.com   2022-09-23   
72    Bryan, Mr.     Hoffman, Tina   mr..bryan@abcfoodmart.com   2023-08-26   

                           Store       Shift Start         Shift End  \
0   ABC Foodmart - Staten Island  2023-04-14 00:00  2023-04-14 00:00   
49     ABC Foodmart - Whitestone  2022-10-04 00:00  2022-10-04 00:00   
50        ABC Foodmart - Tribeca  2022-09-22 00:00  2022-09-22 00:00   
64        ABC Foodmart - Tribeca  2022-10-28 00:00  2022-10-28 00:00   
72  ABC Foodmart - Staten Island  2023-08-28 00:00  2023-08-28 00:00   

          Station  Hourly Pay Rate  store_id first_name last_name  
0         seafood       

In [134]:
#Now, since we have parse the dataframe that includes the employees without last name
#We will parse another dataframe with employees that have first and last name
df2_no_prefix = employee_df[~employee_df['Employee Name'].str.contains(r',\s*Mr\.|Ms\.|Dr\.|Mrs\.', regex=True)]
print(df2_no_prefix.head())
print(len(df2_no_prefix))

        Employee Name    Manager Name                              Email  \
1    Foster, Jennifer   Hoffman, Tina    jennifer.foster@abcfoodmart.com   
2        Shah, Amanda    Ellis, Grant        amanda.shah@abcfoodmart.com   
3       Garcia, Sarah  Cooper, Robert       sarah.garcia@abcfoodmart.com   
4       Riley, Ashley   Hoffman, Tina       ashley.riley@abcfoodmart.com   
5  Campbell, Brittany  Flores, Angela  brittany.campbell@abcfoodmart.com   

  Date of Hire                   Store       Shift Start         Shift End  \
1   2023-06-15    ABC Foodmart - DUMBO  2023-07-08 00:00  2023-07-08 00:00   
2   2023-10-07    ABC Foodmart - DUMBO  2023-10-18 00:00  2023-10-18 00:00   
3   2022-12-14  ABC Foodmart - Tribeca  2023-01-06 00:00  2023-01-06 00:00   
4   2022-08-17    ABC Foodmart - DUMBO  2022-08-27 00:00  2022-08-27 00:00   
5   2023-03-25  ABC Foodmart - Tribeca  2023-04-23 00:00  2023-04-23 00:00   

            Station  Hourly Pay Rate  store_id  
1         dry goods      

In [135]:
import warnings
warnings.filterwarnings('ignore')
df2_no_prefix[['last_name', 'first_name']] = df2_no_prefix['Employee Name'].apply(
    lambda x: pd.Series(x.split(', ')) if x.count(',') == 1 else pd.Series([None, None])
)

In [136]:
print(df2_no_prefix.head())


        Employee Name    Manager Name                              Email  \
1    Foster, Jennifer   Hoffman, Tina    jennifer.foster@abcfoodmart.com   
2        Shah, Amanda    Ellis, Grant        amanda.shah@abcfoodmart.com   
3       Garcia, Sarah  Cooper, Robert       sarah.garcia@abcfoodmart.com   
4       Riley, Ashley   Hoffman, Tina       ashley.riley@abcfoodmart.com   
5  Campbell, Brittany  Flores, Angela  brittany.campbell@abcfoodmart.com   

  Date of Hire                   Store       Shift Start         Shift End  \
1   2023-06-15    ABC Foodmart - DUMBO  2023-07-08 00:00  2023-07-08 00:00   
2   2023-10-07    ABC Foodmart - DUMBO  2023-10-18 00:00  2023-10-18 00:00   
3   2022-12-14  ABC Foodmart - Tribeca  2023-01-06 00:00  2023-01-06 00:00   
4   2022-08-17    ABC Foodmart - DUMBO  2022-08-27 00:00  2022-08-27 00:00   
5   2023-03-25  ABC Foodmart - Tribeca  2023-04-23 00:00  2023-04-23 00:00   

            Station  Hourly Pay Rate  store_id last_name first_name  
1   

In [137]:
#Now merge the two dataframe together to use for further steps
df2_new = pd.concat([df2_with_prefixes,df2_no_prefix])

print(df2_new.head())
print(len(df2_new))

   Employee Name      Manager Name                       Email Date of Hire  \
0      Sean, Mr.  Compton, Jeffrey    mr..sean@abcfoodmart.com   2023-03-25   
49   Terri, Mrs.       Moran, John  mrs..terri@abcfoodmart.com   2022-09-23   
50     Eric, Mr.      Waller, Chad    mr..eric@abcfoodmart.com   2022-09-02   
64   Terri, Mrs.        David, Mr.  mrs..terri@abcfoodmart.com   2022-09-23   
72    Bryan, Mr.     Hoffman, Tina   mr..bryan@abcfoodmart.com   2023-08-26   

                           Store       Shift Start         Shift End  \
0   ABC Foodmart - Staten Island  2023-04-14 00:00  2023-04-14 00:00   
49     ABC Foodmart - Whitestone  2022-10-04 00:00  2022-10-04 00:00   
50        ABC Foodmart - Tribeca  2022-09-22 00:00  2022-09-22 00:00   
64        ABC Foodmart - Tribeca  2022-10-28 00:00  2022-10-28 00:00   
72  ABC Foodmart - Staten Island  2023-08-28 00:00  2023-08-28 00:00   

          Station  Hourly Pay Rate  store_id first_name last_name  
0         seafood       

In [139]:
##There are 100 unique employees based on unique emails
unique_emails = employee_df['Email'].unique()
print(len(employee_df['Email'].unique()))

100


In [140]:
#We need to create employee id for these 100 unique employees for future steps
df_employees = pd.DataFrame({'employee_email': unique_emails})
df_employees.insert(0, 'employee_id', range(1, 1 + len(df_employees)))

print(df_employees.head())

   employee_id                   employee_email
0            1         mr..sean@abcfoodmart.com
1            2  jennifer.foster@abcfoodmart.com
2            3      amanda.shah@abcfoodmart.com
3            4     sarah.garcia@abcfoodmart.com
4            5     ashley.riley@abcfoodmart.com


In [141]:
##Mapping employee_id to each employee in the main dataframe
employee_id_list = [df_employees.employee_id[df_employees.employee_email == i].values[0] for i in df2_new.Email]

#add the employee_id into the main dataframe
df2_new.insert(0, 'employee_id', employee_id_list)

print(df2_new.head())

    employee_id Employee Name      Manager Name                       Email  \
0             1     Sean, Mr.  Compton, Jeffrey    mr..sean@abcfoodmart.com   
49           40   Terri, Mrs.       Moran, John  mrs..terri@abcfoodmart.com   
50           41     Eric, Mr.      Waller, Chad    mr..eric@abcfoodmart.com   
64           40   Terri, Mrs.        David, Mr.  mrs..terri@abcfoodmart.com   
72           54    Bryan, Mr.     Hoffman, Tina   mr..bryan@abcfoodmart.com   

   Date of Hire                         Store       Shift Start  \
0    2023-03-25  ABC Foodmart - Staten Island  2023-04-14 00:00   
49   2022-09-23     ABC Foodmart - Whitestone  2022-10-04 00:00   
50   2022-09-02        ABC Foodmart - Tribeca  2022-09-22 00:00   
64   2022-09-23        ABC Foodmart - Tribeca  2022-10-28 00:00   
72   2023-08-26  ABC Foodmart - Staten Island  2023-08-28 00:00   

           Shift End        Station  Hourly Pay Rate  store_id first_name  \
0   2023-04-14 00:00        seafood          

In [142]:
#100 employees should link to 100 unique employee_id in total
unique_employee_ids = df2_new['employee_id'].nunique()

print(unique_employee_ids)

100


In [143]:
print(df2_new[df2_new['employee_id']==1].head())

     employee_id Employee Name      Manager Name                     Email  \
0              1     Sean, Mr.  Compton, Jeffrey  mr..sean@abcfoodmart.com   
142            1     Sean, Mr.    Cooper, Robert  mr..sean@abcfoodmart.com   
171            1     Sean, Mr.    Cooper, Robert  mr..sean@abcfoodmart.com   
194            1     Sean, Mr.  Compton, Jeffrey  mr..sean@abcfoodmart.com   
244            1     Sean, Mr.  Compton, Jeffrey  mr..sean@abcfoodmart.com   

    Date of Hire                         Store       Shift Start  \
0     2023-03-25  ABC Foodmart - Staten Island  2023-04-14 00:00   
142   2023-03-25  ABC Foodmart - Staten Island  2023-04-24 00:00   
171   2023-03-25  ABC Foodmart - Staten Island  2023-05-08 00:00   
194   2023-03-25  ABC Foodmart - Staten Island  2023-05-20 00:00   
244   2023-03-25          ABC Foodmart - DUMBO  2023-06-05 00:00   

            Shift End       Station  Hourly Pay Rate  store_id first_name  \
0    2023-04-14 00:00       seafood          

In [144]:
#Convert the Shift End into pandas Date format
df2_new['time_shift_end'] = pd.to_datetime(df2_new['Shift End'])

print(df2_new['time_shift_end'].head())

0    2023-04-14
49   2022-10-04
50   2022-09-22
64   2022-10-28
72   2023-08-28
Name: time_shift_end, dtype: datetime64[ns]


In [145]:
#Since our schema only allows One employee per position/salary/store at a time, we will choose the data with
#the latest "Shift End" for each unique employee

# Sort it by 'time_shift_end' in descending order
df_sorted = df2_new.sort_values(by='time_shift_end', ascending=False)

# Drop duplicates based on 'Email', keep the most recent
df_unique_employees = df_sorted.drop_duplicates(subset='Email', keep='first')

In [146]:
print(len(df_unique_employees))
print(df_unique_employees.head())

100
      employee_id   Employee Name    Manager Name  \
4944            9    Walters, Jay   Hoffman, Tina   
4975           20  Martin, Monica  Cooper, Robert   
4820           32    Love, Thomas   Carter, Karen   
4902           78  Horton, Ashley     Moran, John   
4994           37  Church, Joshua  Cooper, Robert   

                              Email Date of Hire  \
4944    jay.walters@abcfoodmart.com   2023-12-17   
4975  monica.martin@abcfoodmart.com   2023-11-23   
4820    thomas.love@abcfoodmart.com   2024-02-18   
4902  ashley.horton@abcfoodmart.com   2024-01-24   
4994  joshua.church@abcfoodmart.com   2024-03-03   

                             Store       Shift Start         Shift End  \
4944          ABC Foodmart - DUMBO  2026-09-07 00:00  2026-09-07 00:00   
4975      ABC Foodmart - Bay Ridge  2026-08-25 00:00  2026-08-25 00:00   
4820  ABC Foodmart - Staten Island  2026-08-06 00:00  2026-08-06 00:00   
4902      ABC Foodmart - Bay Ridge  2026-07-09 00:00  2026-07-09 00:

In [147]:
#We will keep the df_unique_employees when the Sales Table reqwuire aditional information

#We will also create the employee Table right now since there is no complex relationships
df_unique_employees.rename(columns={'Station': 'position','Hourly Pay Rate':'salary'}, inplace = True)

complete_employees_df = df_unique_employees[['employee_id','first_name','last_name','position','store_id','salary']]

print(complete_employees_df.head())

# Check for None values in all columns
any_none_values = complete_employees_df.isna().any()

print(any_none_values)

      employee_id first_name last_name          position  store_id  salary
4944            9        Jay   Walters           cashier         1   18.98
4975           20     Monica    Martin            bakery         3   15.59
4820           32     Thomas      Love            floral         5   12.72
4902           78     Ashley    Horton          pharmacy         3   16.58
4994           37     Joshua    Church  customer service         3   12.93
employee_id    False
first_name     False
last_name       True
position       False
store_id       False
salary         False
dtype: bool


In [148]:
complete_employees_df.to_csv("Employee Table Insertion.csv", index=False)

#### Create OperatingCost Table From Scratch before joining tables
- Since OperatingCost works in Store table
- It is important to generate the dataframe for the OperatingCost Table 
- Use the stores.csv for data manipulation
- Map the store_id with the dataframe created in previous steps

In [149]:
opcost = pd.read_csv('stores.csv')
print(opcost.head())
print(len(opcost))

             store_name    store_street store_city store_state  store_zipcode  \
0  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
1  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
2  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
3  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   
4  ABC Foodmart - DUMBO  123 Jay Street   Brooklyn          NY          11201   

   store_manager store_open_date  store_parking_capacity store_expense_date  \
0  Hoffman, Tina      2017-06-18                      50         2024-04-01   
1  Hoffman, Tina      2017-06-18                      50         2024-03-01   
2  Hoffman, Tina      2017-06-18                      50         2024-02-01   
3  Hoffman, Tina      2017-06-18                      50         2024-01-01   
4  Hoffman, Tina      2017-06-18                      50         2023-12-01   

  store_expense_type  store_expense_am

In [150]:
#Map the store id before data manipulation

#Create a dictionary mapping store names to store_id
store_mapping = store_table_df.set_index('store_name')['store_id'].to_dict()

#Map store_name to store_id in opcost
opcost['store_id'] = opcost['store_name'].map(store_mapping)

print(opcost)

                       store_name       store_street     store_city  \
0            ABC Foodmart - DUMBO     123 Jay Street       Brooklyn   
1            ABC Foodmart - DUMBO     123 Jay Street       Brooklyn   
2            ABC Foodmart - DUMBO     123 Jay Street       Brooklyn   
3            ABC Foodmart - DUMBO     123 Jay Street       Brooklyn   
4            ABC Foodmart - DUMBO     123 Jay Street       Brooklyn   
..                            ...                ...            ...   
395  ABC Foodmart - Staten Island  2655 Richmond Ave  Staten Island   
396  ABC Foodmart - Staten Island  2655 Richmond Ave  Staten Island   
397  ABC Foodmart - Staten Island  2655 Richmond Ave  Staten Island   
398  ABC Foodmart - Staten Island  2655 Richmond Ave  Staten Island   
399  ABC Foodmart - Staten Island  2655 Richmond Ave  Staten Island   

    store_state  store_zipcode         store_manager store_open_date  \
0            NY          11201         Hoffman, Tina      2017-06-18   
1  

In [151]:
#Verify Mapping
#Create a set of unique store_id from the store_table_df
store_ids_store_table = set(store_table_df['store_id'])

#Create a set of unique store_id from the employee_df
store_ids_opcost = set(opcost['store_id'])

#Check if the sets of unique store IDs are equal
if store_ids_store_table == store_ids_opcost:
    print("The mapping is the same for both tables.")
else:
    print("The mapping is different for both tables.")

The mapping is the same for both tables.


In [152]:
grouped_df = opcost.groupby(['store_expense_date','store_id'])['store_expense_description'].agg(lambda x: ', '.join(x)).reset_index()
print(grouped_df.head())

  store_expense_date  store_id                      store_expense_description
0         2021-01-01         1  January rent payment, gas, water, electricity
1         2021-01-01         2  January rent payment, gas, water, electricity
2         2021-01-01         3  January rent payment, gas, water, electricity
3         2021-01-01         4  January rent payment, gas, water, electricity
4         2021-01-01         5  January rent payment, gas, water, electricity


In [154]:
#Since our OperatingCost includes fixed cost and variable cost, we need to add the value based on descriptions
opcost['variable_cost'] = opcost.apply(lambda x: x['store_expense_amount'] if x['store_expense_type'] == 'utilities' else 0, axis=1)
opcost['fixed_cost'] = opcost.apply(lambda x: x['store_expense_amount'] if x['store_expense_type'] == 'rent' else 0, axis=1)
print(opcost[['variable_cost', 'fixed_cost']].head())

   variable_cost  fixed_cost
0              0        6500
1              0        6500
2              0        6500
3              0        6500
4              0        6000


In [155]:
#Since different stores have cost on same days, we need to calculate the sum based on this
grouped_df2 = opcost.groupby(['store_expense_date','store_id']).agg({
    'variable_cost': 'sum',
    'fixed_cost': 'sum'
}).reset_index()
print(grouped_df2.head())

  store_expense_date  store_id  variable_cost  fixed_cost
0         2021-01-01         1           1628        5000
1         2021-01-01         2           1875        9000
2         2021-01-01         3           1353        8250
3         2021-01-01         4           1163       14500
4         2021-01-01         5           1159       10000


In [156]:
merged_df = pd.merge(grouped_df, grouped_df2, on=['store_expense_date', 'store_id'], how='inner')
print(merged_df.head())

  store_expense_date  store_id                      store_expense_description  \
0         2021-01-01         1  January rent payment, gas, water, electricity   
1         2021-01-01         2  January rent payment, gas, water, electricity   
2         2021-01-01         3  January rent payment, gas, water, electricity   
3         2021-01-01         4  January rent payment, gas, water, electricity   
4         2021-01-01         5  January rent payment, gas, water, electricity   

   variable_cost  fixed_cost  
0           1628        5000  
1           1875        9000  
2           1353        8250  
3           1163       14500  
4           1159       10000  


In [157]:
#Based on date and the unique store_id, the table includes 200 rows of costs records
print(len(merged_df))

200


In [158]:
#We need to create a table for the OperatingCost Table for data insertion
operatingcost_df = merged_df.copy()

print(operatingcost_df.head())
print(len(operatingcost_df))

  store_expense_date  store_id                      store_expense_description  \
0         2021-01-01         1  January rent payment, gas, water, electricity   
1         2021-01-01         2  January rent payment, gas, water, electricity   
2         2021-01-01         3  January rent payment, gas, water, electricity   
3         2021-01-01         4  January rent payment, gas, water, electricity   
4         2021-01-01         5  January rent payment, gas, water, electricity   

   variable_cost  fixed_cost  
0           1628        5000  
1           1875        9000  
2           1353        8250  
3           1163       14500  
4           1159       10000  
200


In [161]:
operatingcost_df = operatingcost_df.rename(columns = {'store_expense_date':'date','store_expense_description':'cost_description'})
merged_df = merged_df.rename(columns = {'store_expense_date':'date','store_expense_description':'cost_description'})

In [162]:
print(operatingcost_df.head())
print(merged_df.head())

         date  store_id                               cost_description  \
0  2021-01-01         1  January rent payment, gas, water, electricity   
1  2021-01-01         2  January rent payment, gas, water, electricity   
2  2021-01-01         3  January rent payment, gas, water, electricity   
3  2021-01-01         4  January rent payment, gas, water, electricity   
4  2021-01-01         5  January rent payment, gas, water, electricity   

   variable_cost  fixed_cost  
0           1628        5000  
1           1875        9000  
2           1353        8250  
3           1163       14500  
4           1159       10000  
         date  store_id                               cost_description  \
0  2021-01-01         1  January rent payment, gas, water, electricity   
1  2021-01-01         2  January rent payment, gas, water, electricity   
2  2021-01-01         3  January rent payment, gas, water, electricity   
3  2021-01-01         4  January rent payment, gas, water, electricity   

In [163]:
#Read into csv file for keep record
operatingcost_df.to_csv("OperatingCost Table Insertion.csv", index=False)

#### Create Sales DataFrame for breaking down tables
- For Sales Table
- For Specific Sales Table
- For Customer Table
- For Customer Rating

Inlcude variables from Other Tables:
- LoyaltyProgram Table
- Payment Table


In [164]:
#Read in the customer_sales dataset
df4 = pd.read_csv('customers_sales.csv')
print(len(df4))
print(df4.head())

22205
  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

   Date of Purchase  Product Name  Quantity  Price per Unit  
0  2023-07-14 00:17         Roses         6          155.82  
1  2023-07-14 00:17    Roast Beef        10          147.62  
2  2023-07-14 00:17  Frozen Pizza        10           26.13  
3  2023-07-14 00:17   Cough Syrup         8           53.08  
4  2023-07-14 00:17       Lettuce         8           83.83  


##### Start with creating Customer Table
- customer_id (needed)
- first_name (need to be break down)
- last_name (need to be break down)
- email (included)
- phone (needed to be generated)
- loyalty_id (needed to be created during we generate the customer numbers for the Table)

In [165]:
#Check for missing values in each column of df4
missing_values = df4.isnull().sum()

print(missing_values)


Customer Name       0
Customer Email      0
Store Name          0
Date of Purchase    0
Product Name        0
Quantity            0
Price per Unit      0
dtype: int64


In [166]:
print(len(df4['Customer Name'].unique()))
print(len(df4['Customer Email'].unique()))
#There could be customer with same name in the dataset
#There could also be customer who changes email throughout all purchases

1966
1993


In [167]:
result = df4.groupby('Customer Name').filter(lambda x: x['Customer Email'].nunique() > 1)
print(result)

         Customer Name            Customer Email              Store Name  \
125     Michael Morris  sandraarnold@example.org    ABC Foodmart - DUMBO   
126     Michael Morris  sandraarnold@example.org    ABC Foodmart - DUMBO   
127     Michael Morris  sandraarnold@example.org    ABC Foodmart - DUMBO   
128     Michael Morris  sandraarnold@example.org    ABC Foodmart - DUMBO   
191    Stephen Roberts        mark02@example.org  ABC Foodmart - Tribeca   
...                ...                       ...                     ...   
21623   Ashley Morales       wmiller@example.com  ABC Foodmart - Tribeca   
21624   Ashley Morales       wmiller@example.com  ABC Foodmart - Tribeca   
21625   Ashley Morales       wmiller@example.com  ABC Foodmart - Tribeca   
21626   Ashley Morales       wmiller@example.com  ABC Foodmart - Tribeca   
21627   Ashley Morales       wmiller@example.com  ABC Foodmart - Tribeca   

       Date of Purchase   Product Name  Quantity  Price per Unit  
125    2023-09-05 08

In [172]:
#We will assume that some customer changes email mid way
#Create customer_id for each unique customer
df_customer = df4['Customer Email'].unique()

df_customer = pd.DataFrame({'Customer Email': df_customer})

print(df_customer)


                  Customer Email
0              ucook@example.net
1        stephanie93@example.com
2           dsanchez@example.net
3      vickimitchell@example.com
4        sabrinawade@example.org
...                          ...
1988         duane09@example.com
1989       ucastillo@example.net
1990  tammyhernandez@example.org
1991   jodirodriguez@example.com
1992         tammy28@example.org

[1993 rows x 1 columns]


In [173]:
df_customer.insert(0, 'customer_id', range(1, 1 + len(df_customer)))

In [174]:
print(df_customer.head())

   customer_id             Customer Email
0            1          ucook@example.net
1            2    stephanie93@example.com
2            3       dsanchez@example.net
3            4  vickimitchell@example.com
4            5    sabrinawade@example.org


In [175]:
##Mapping customer_id to each customer name in the main dataframe
customer_id_list = [df_customer.customer_id[df_customer['Customer Email'] == i].values[0] for i in df4['Customer Email']]

#add the customer_id into the main dataframe
df4.insert(5, 'customer_id', customer_id_list)

In [179]:
print(df4.head())

  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

   Date of Purchase  Product Name  customer_id  Quantity  Price per Unit  
0  2023-07-14 00:17         Roses            1         6          155.82  
1  2023-07-14 00:17    Roast Beef            1        10          147.62  
2  2023-07-14 00:17  Frozen Pizza            1        10           26.13  
3  2023-07-14 00:17   Cough Syrup            1         8           53.08  
4  2023-07-14 00:17       Lettuce            1         8           83.83  


In [180]:
#Now we want to select the unique customer to create a dataframe for recording Customer values
df_unique_customers = df4.drop_duplicates(subset=['Customer Email'], keep='first')

print(len(df_unique_customers['Customer Email'].unique()))

1993


In [181]:
#Remove prefix and post fix and assign first and last name
import re
def clean_name(name):
    name = re.sub(r'^(Mr\.|Mrs\.|Ms\.|Dr\.)\s+', '', name)
    
    name = re.sub(r'\s+(MD|DVM|PhD|DDS|Jr\.|Sr\.)$', '', name)
    
    # Split the remaining string by the first space to separate first and last names
    parts = name.split(maxsplit=1)
    first_name = parts[0]
    last_name = parts[1] if len(parts) > 1 else ''  # Some names might not have a last name
    
    return first_name, last_name


df_unique_customers[['first_name', 'last_name']] = df_unique_customers['Customer Name'].apply(lambda x: pd.Series(clean_name(x)))

In [182]:
print(df_unique_customers.head())

        Customer Name             Customer Email                 Store Name  \
0         Kathy Jones          ucook@example.net   ABC Foodmart - Bay Ridge   
13       Richard Diaz    stephanie93@example.com     ABC Foodmart - Tribeca   
25      Steven Medina       dsanchez@example.net       ABC Foodmart - DUMBO   
38  Angela Washington  vickimitchell@example.com   ABC Foodmart - Bay Ridge   
47   Valerie Crawford    sabrinawade@example.org  ABC Foodmart - Whitestone   

    Date of Purchase Product Name  customer_id  Quantity  Price per Unit  \
0   2023-07-14 00:17        Roses            1         6          155.82   
13  2023-08-03 11:28       Apples            2         7          136.67   
25  2023-05-11 00:50         Rice            3         7          147.47   
38  2023-05-14 12:29   Beef Steak            4         4           62.98   
47  2023-10-15 10:40        Juice            5         5           32.57   

   first_name   last_name  
0       Kathy       Jones  
13    Richar

##### Create Loyalty Program (not all customer need loyalty program)
- loyalty_id needed
- level needed
- points needed

In [183]:
# Define the parameters for the DataFrame
num_rows = len(df_unique_customers)
levels = ['silver', 'gold', 'platinum']
points_range = (20, 10000)

# Generate the data
np.random.seed(1310) 
loyalty_id = np.arange(1, num_rows + 1) 
level = np.random.choice(levels, num_rows)  
points = np.random.randint(points_range[0], points_range[1] + 1, num_rows) 

loyalty_program = pd.DataFrame({
    'loyalty_id': loyalty_id,
    'level': level,
    'points': points
})

# Display the first few rows 
loyalty_program.head()

,loyalty_id,level,points
0,1,silver,5958
1,2,silver,8001
2,3,silver,9601
3,4,gold,4926
4,5,platinum,3644


In [184]:
#This loyaltyprogram dataframe will be used directly for the LoyaltyProgram Insertion
loyalty_program.to_csv("LoyaltyProgram Table Insertion.csv", index = False)

In [185]:
# Create phone numbers
# Define the number of phone numbers
num_numbers = len(df_unique_customers)

phone_numbers = [''.join([str(np.random.randint(1, 10))] + ['%i' % np.random.randint(0, 10) for j in range(9)]) for i in range(num_numbers)]
phone_numbers = [int(number) for number in phone_numbers]  # Convert to integers

phone_df = pd.DataFrame({'phone_number': phone_numbers})

print(phone_df.head())

   phone_number
0    8243343078
1    2578799891
2    3932152834
3    6794472057
4    3886602382


In [186]:
#Put phone data with customer data
#df_unique_customers['phone'] = phone_df['phone_number']
df_unique_customers = df_unique_customers.assign(phone=phone_df['phone_number'].values)

In [187]:
print(df_unique_customers.tail())

          Customer Name              Customer Email                Store Name  \
22135  Timothy Campbell         duane09@example.com  ABC Foodmart - Bay Ridge   
22152       Scott Ayala       ucastillo@example.net  ABC Foodmart - Bay Ridge   
22171      Zachary Rios  tammyhernandez@example.org  ABC Foodmart - Bay Ridge   
22184    Caroline Smith   jodirodriguez@example.com      ABC Foodmart - DUMBO   
22194     Steven Reilly         tammy28@example.org  ABC Foodmart - Bay Ridge   

       Date of Purchase  Product Name  customer_id  Quantity  Price per Unit  \
22135  2023-12-28 19:12          Rice         1989         1          155.53   
22152  2023-11-28 06:26    Beef Steak         1990         5            9.95   
22171  2023-09-08 19:57     Croissant         1991         2          177.50   
22184  2023-12-14 06:04          Milk         1992         2           61.28   
22194  2023-09-28 13:35  Frozen Pizza         1993         6           86.76   

      first_name last_name      

In [188]:
##Put loyalty data with customer data
df_unique_customers = df_unique_customers.assign(loyalty_id=loyalty_program['loyalty_id'].values)
df_unique_customers = df_unique_customers.assign(level=loyalty_program['level'].values)
df_unique_customers = df_unique_customers.assign(points=loyalty_program['points'].values)

In [189]:
#Check if there is null value
missing_values_unique_customers = df_unique_customers.isnull().sum()
print(missing_values_unique_customers)

Customer Name       0
Customer Email      0
Store Name          0
Date of Purchase    0
Product Name        0
customer_id         0
Quantity            0
Price per Unit      0
first_name          0
last_name           0
phone               0
loyalty_id          0
level               0
points              0
dtype: int64


In [190]:
#We need to keep the df_unique_customers for matching the sales table in df4
#We will generate a spearate table for Customer Table Insertion
df_customer_selected = df_unique_customers[['customer_id','first_name','last_name','phone','Customer Email','loyalty_id']]
df_customer_selected = df_customer_selected.rename(columns = {"Customer Email":"email"})
print(df_customer_selected.head())

    customer_id first_name   last_name       phone                      email  \
0             1      Kathy       Jones  8243343078          ucook@example.net   
13            2    Richard        Diaz  2578799891    stephanie93@example.com   
25            3     Steven      Medina  3932152834       dsanchez@example.net   
38            4     Angela  Washington  6794472057  vickimitchell@example.com   
47            5    Valerie    Crawford  3886602382    sabrinawade@example.org   

    loyalty_id  
0            1  
13           2  
25           3  
38           4  
47           5  


In [191]:
df_customer_selected.to_csv("Customer Table Insertion.csv", index = False)

##### Create CustomerRating Table
- Import customer review dataset
- Create customer review for some of the customers
- Make sure the review is made after the purchase

In [192]:
raw_review = pd.read_csv("sample_reviews.csv", sep = '/')
print(raw_review.head())

   Rating                                        Description
0     5.0                5, great selection of fresh produce
1     3.0  5, a well-stocked supermarket with friendly staff
2     3.0                      5, good for one-stop shopping
3     1.0                                  5, not very clean
4     5.0                5, great selection of fresh produce


In [193]:
raw_review['Description'] = raw_review['Description'].apply(lambda x: x.split(',', 1)[1].strip() if ',' in x else x)
print(raw_review.head())
print(len(raw_review))

   Rating                                     Description
0     5.0                great selection of fresh produce
1     3.0  a well-stocked supermarket with friendly staff
2     3.0                      good for one-stop shopping
3     1.0                                  not very clean
4     5.0                great selection of fresh produce
133


In [194]:
##Add these reviews to the first 133 customers 
df_unique_customers_selected = df_unique_customers[0:133]
print(df_unique_customers_selected.head())

        Customer Name             Customer Email                 Store Name  \
0         Kathy Jones          ucook@example.net   ABC Foodmart - Bay Ridge   
13       Richard Diaz    stephanie93@example.com     ABC Foodmart - Tribeca   
25      Steven Medina       dsanchez@example.net       ABC Foodmart - DUMBO   
38  Angela Washington  vickimitchell@example.com   ABC Foodmart - Bay Ridge   
47   Valerie Crawford    sabrinawade@example.org  ABC Foodmart - Whitestone   

    Date of Purchase Product Name  customer_id  Quantity  Price per Unit  \
0   2023-07-14 00:17        Roses            1         6          155.82   
13  2023-08-03 11:28       Apples            2         7          136.67   
25  2023-05-11 00:50         Rice            3         7          147.47   
38  2023-05-14 12:29   Beef Steak            4         4           62.98   
47  2023-10-15 10:40        Juice            5         5           32.57   

   first_name   last_name       phone  loyalty_id     level  points 

In [195]:
#Add customer reviews data into the uniquie customer frame
df_unique_customers_selected = df_unique_customers_selected.assign(rating=raw_review['Rating'].values)
df_unique_customers_selected = df_unique_customers_selected.assign(review=raw_review['Description'].values)
print(df_unique_customers_selected.head())

        Customer Name             Customer Email                 Store Name  \
0         Kathy Jones          ucook@example.net   ABC Foodmart - Bay Ridge   
13       Richard Diaz    stephanie93@example.com     ABC Foodmart - Tribeca   
25      Steven Medina       dsanchez@example.net       ABC Foodmart - DUMBO   
38  Angela Washington  vickimitchell@example.com   ABC Foodmart - Bay Ridge   
47   Valerie Crawford    sabrinawade@example.org  ABC Foodmart - Whitestone   

    Date of Purchase Product Name  customer_id  Quantity  Price per Unit  \
0   2023-07-14 00:17        Roses            1         6          155.82   
13  2023-08-03 11:28       Apples            2         7          136.67   
25  2023-05-11 00:50         Rice            3         7          147.47   
38  2023-05-14 12:29   Beef Steak            4         4           62.98   
47  2023-10-15 10:40        Juice            5         5           32.57   

   first_name   last_name       phone  loyalty_id     level  points 

In [196]:
print(len(df_unique_customers_selected))

133


In [197]:
#Mapping store_id to each store in the store_table_df
#Create a dictionary mapping store names to store_id
store_mapping = store_table_df.set_index('store_name')['store_id'].to_dict()

#Map store_name to store_id in df_unique_customers_selected
df_unique_customers_selected['store_id'] = df_unique_customers_selected['Store Name'].map(store_mapping)

print(df_unique_customers_selected)

          Customer Name                  Customer Email  \
0           Kathy Jones               ucook@example.net   
13         Richard Diaz         stephanie93@example.com   
25        Steven Medina            dsanchez@example.net   
38    Angela Washington       vickimitchell@example.com   
47     Valerie Crawford         sabrinawade@example.org   
...                 ...                             ...   
1459       Daniel Moore       mitchelljacob@example.org   
1478      Michael Lewis              ocombs@example.com   
1489      Tristan Smith            mayjenna@example.net   
1508       David Harper         warnerbrett@example.net   
1519   Shannon Williams  christopherswanson@example.com   

                     Store Name  Date of Purchase Product Name  customer_id  \
0      ABC Foodmart - Bay Ridge  2023-07-14 00:17        Roses            1   
13       ABC Foodmart - Tribeca  2023-08-03 11:28       Apples            2   
25         ABC Foodmart - DUMBO  2023-05-11 00:50     

In [198]:
#create a "date of review" which must be later than date of sale

df_unique_customers_selected['Date of Purchase_temp'] = pd.to_datetime(df_unique_customers_selected['Date of Purchase'])

# Generate random days offsets, ensuring they are always greater than 0
np.random.seed(1031)  # For reproducibility
random_days = np.random.randint(0, 31, size=len(df_unique_customers_selected))  # Random days between 0 and 30

# Add the random number of days to 'date_of_purchase'
df_unique_customers_selected['review_date'] = df_unique_customers_selected['Date of Purchase_temp'] + pd.to_timedelta(random_days, unit='D')

print(df_unique_customers_selected.head())

        Customer Name             Customer Email                 Store Name  \
0         Kathy Jones          ucook@example.net   ABC Foodmart - Bay Ridge   
13       Richard Diaz    stephanie93@example.com     ABC Foodmart - Tribeca   
25      Steven Medina       dsanchez@example.net       ABC Foodmart - DUMBO   
38  Angela Washington  vickimitchell@example.com   ABC Foodmart - Bay Ridge   
47   Valerie Crawford    sabrinawade@example.org  ABC Foodmart - Whitestone   

    Date of Purchase Product Name  customer_id  Quantity  Price per Unit  \
0   2023-07-14 00:17        Roses            1         6          155.82   
13  2023-08-03 11:28       Apples            2         7          136.67   
25  2023-05-11 00:50         Rice            3         7          147.47   
38  2023-05-14 12:29   Beef Steak            4         4           62.98   
47  2023-10-15 10:40        Juice            5         5           32.57   

   first_name   last_name       phone  loyalty_id     level  points 

In [199]:
#Generate customer rating based on the result, and use it for CustomerRating Table Insertion
complete_review = df_unique_customers_selected[['customer_id','store_id','review_date','rating','review']]
print(complete_review.head())

    customer_id  store_id         review_date  rating  \
0             1         3 2023-08-04 00:17:00     5.0   
13            2         2 2023-08-27 11:28:00     3.0   
25            3         1 2023-05-11 00:50:00     3.0   
38            4         3 2023-06-06 12:29:00     1.0   
47            5         4 2023-11-14 10:40:00     5.0   

                                            review  
0                 great selection of fresh produce  
13  a well-stocked supermarket with friendly staff  
25                      good for one-stop shopping  
38                                  not very clean  
47                great selection of fresh produce  


In [200]:
missing_values_rating = complete_review.isna().sum()
print(missing_values_rating)

customer_id    0
store_id       0
review_date    0
rating         0
review         0
dtype: int64


In [201]:
complete_review.to_csv("CustomerRating Table Insertion.csv", index = False)

##### Create Sales Table for data Insertion
- Based on the customers_sales.csv
- Mapping to previous df_unique_customers for customer_id insertion
- Mapping to store_table_df for store_id insertion
- Mapping to complete_employees_df for employee_id insertion

In [209]:
customers_sales= pd.read_csv("customers_sales.csv")
print(customers_sales.head())

  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

   Date of Purchase  Product Name  Quantity  Price per Unit  
0  2023-07-14 00:17         Roses         6          155.82  
1  2023-07-14 00:17    Roast Beef        10          147.62  
2  2023-07-14 00:17  Frozen Pizza        10           26.13  
3  2023-07-14 00:17   Cough Syrup         8           53.08  
4  2023-07-14 00:17       Lettuce         8           83.83  


In [214]:
customers_sales['Date of Purchase'] = pd.to_datetime(customers_sales['Date of Purchase'])

In [215]:
#For the sales_id, we need to create the id based on the time a customer purchase
# and the customer name him/her self
#This ensure that at one transaction time, different customer will have different Sales id
# Combine 'Customer Name' and 'Date of Purchase' to create a unique identifier
# Group by 'Customer Name', 'Date of Purchase', and 'Store Name' and assign a single sales_id to each group

#Initialize a dictionary to store the mapping of (Customer Name, Date of Purchase) to sales_id
sales_id_mapping = {}

#Iterate through the rows of the DataFrame
for index, row in customers_sales.iterrows():
    # Get the values of 'Customer Name' and 'Date of Purchase' for the current row
    customer_name = row['Customer Name']
    purchase_date = row['Date of Purchase']
    
    #Check if the (Customer Name, Date of Purchase) combination already exists in the mapping
    if (customer_name, purchase_date) not in sales_id_mapping:
        # If not, assign a new sales_id and update the mapping
        sales_id_mapping[(customer_name, purchase_date)] = len(sales_id_mapping) + 1
    
    #Assign the sales_id to the current row
    customers_sales.at[index, 'sales_id'] = sales_id_mapping[(customer_name, purchase_date)]

print(customers_sales)

       Customer Name       Customer Email                Store Name  \
0        Kathy Jones    ucook@example.net  ABC Foodmart - Bay Ridge   
1        Kathy Jones    ucook@example.net  ABC Foodmart - Bay Ridge   
2        Kathy Jones    ucook@example.net  ABC Foodmart - Bay Ridge   
3        Kathy Jones    ucook@example.net  ABC Foodmart - Bay Ridge   
4        Kathy Jones    ucook@example.net  ABC Foodmart - Bay Ridge   
...              ...                  ...                       ...   
22200  Steven Reilly  tammy28@example.org  ABC Foodmart - Bay Ridge   
22201  Steven Reilly  tammy28@example.org  ABC Foodmart - Bay Ridge   
22202  Steven Reilly  tammy28@example.org  ABC Foodmart - Bay Ridge   
22203  Steven Reilly  tammy28@example.org  ABC Foodmart - Bay Ridge   
22204  Steven Reilly  tammy28@example.org  ABC Foodmart - Bay Ridge   

         Date of Purchase  Product Name  Quantity  Price per Unit  sales_id  
0     2023-07-14 00:17:00         Roses         6          155.82    

In [219]:
#Verify whether the sales_id assignment is successful
#Count the number of unique sales_id
unique_sales_ids = customers_sales['sales_id'].nunique()

print("Number of unique sales_id:", unique_sales_ids)

Number of unique sales_id: 2000


In [218]:
#Check one sales_id for verification
#Filter rows where Customer Name is 'Kathy Jones' and Date of Purchase is '2023-07-14 00:17:00'
kathy_jones_purchases = customers_sales[(customers_sales['Customer Name'] == 'Kathy Jones') & 
                                       (customers_sales['Date of Purchase'] == '2023-07-14 00:17:00')]

print(kathy_jones_purchases)

   Customer Name     Customer Email                Store Name  \
0    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
5    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
6    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
7    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
8    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
9    Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
10   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
11   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
12   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

      Date of Purchase       Product Name  Quantity  Price per Unit  sales_id  
0  2023-0

In [220]:
column_names_customers_sales = customers_sales.columns
print(column_names_customers_sales)

Index(['Customer Name', 'Customer Email', 'Store Name', 'Date of Purchase',
       'Product Name', 'Quantity', 'Price per Unit', 'sales_id'],
      dtype='object')


In [233]:
#For Sales Table, we still need customer_id, employee_id, store_id
#First, map the customer_id from the df_unique_customers_selected dataframe
#Create a dictionary mapping Customer Email to customer_id
customer_email_mapping = df_customer_selected.set_index('email')['customer_id'].to_dict()

#Map Customer Email to customer_id in the customers_sales DataFrame
customers_sales['customer_id'] = customers_sales['Customer Email'].map(customer_email_mapping)

print(customers_sales.head())


  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

     Date of Purchase  Product Name  Quantity  Price per Unit  sales_id  \
0 2023-07-14 00:17:00         Roses         6          155.82         1   
1 2023-07-14 00:17:00    Roast Beef        10          147.62         1   
2 2023-07-14 00:17:00  Frozen Pizza        10           26.13         1   
3 2023-07-14 00:17:00   Cough Syrup         8           53.08         1   
4 2023-07-14 00:17:00       Lettuce         8           83.83         1   

   customer_id  
0            1  
1            1  
2            1  
3            1  
4            1  


In [234]:
missing_customer_ids = customers_sales['customer_id'].isnull().sum()

# Print the number of missing customer_id values
print("Number of missing customer_ids:", missing_customer_ids)


Number of missing customer_ids: 0


In [237]:
#Map the store_id through the store_table_df created from previous manipulation
#Create a dictionary mapping Store Name to store_id
store_mapping = store_table_df.set_index('store_name')['store_id'].to_dict()

#Map Store Name to store_id in the customers_sales DataFrame
customers_sales['store_id'] = customers_sales['Store Name'].map(store_mapping)

print(customers_sales.head())

  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

     Date of Purchase  Product Name  Quantity  Price per Unit  sales_id  \
0 2023-07-14 00:17:00         Roses         6          155.82         1   
1 2023-07-14 00:17:00    Roast Beef        10          147.62         1   
2 2023-07-14 00:17:00  Frozen Pizza        10           26.13         1   
3 2023-07-14 00:17:00   Cough Syrup         8           53.08         1   
4 2023-07-14 00:17:00       Lettuce         8           83.83         1   

   customer_id  store_id  
0            1         3  
1            1         3  
2            1         3  
3            1         3  
4            1         3  


In [252]:
#Before Mapping the employee_id to the customers_sales dataframe
#We should generate the partial Sales Table first for easy employee_id assignment

#select the dcolumns from customers_sales and rename Date of Purchase to sales_date
partial_sales = customers_sales[['sales_id', 'customer_id', 'store_id', 'Date of Purchase']]
partial_sales.rename(columns={'Date of Purchase': 'sales_date'}, inplace=True)

#remove duplicates based on customer_id and sales_date
partial_sales = partial_sales.drop_duplicates(subset=['customer_id', 'sales_date'])

partial_sales.reset_index(drop=True, inplace=True)


print(partial_sales.head())
print(len(partial_sales))


   sales_id  customer_id  store_id          sales_date
0         1            1         3 2023-07-14 00:17:00
1         2            2         2 2023-08-03 11:28:00
2         3            3         1 2023-05-11 00:50:00
3         4            4         3 2023-05-14 12:29:00
4         5            5         4 2023-10-15 10:40:00
2000


In [253]:
#Map the employee_id through the employee who is working in each store in complete_employees_df

#group complete_employees_df by store_id and aggregate employee_id into lists
employee_id_mapping = complete_employees_df.groupby('store_id')['employee_id'].apply(list).to_dict()

#randomly select an employee_id for each store_id
def random_employee_id(store_id):
    if store_id in employee_id_mapping:
        return np.random.choice(employee_id_mapping[store_id])
    else:
        return np.nan

#Map store_id to employee_id in the partial_sales
partial_sales['employee_id'] = partial_sales['store_id'].apply(random_employee_id)

print(partial_sales.head())


   sales_id  customer_id  store_id          sales_date  employee_id
0         1            1         3 2023-07-14 00:17:00           33
1         2            2         2 2023-08-03 11:28:00           28
2         3            3         1 2023-05-11 00:50:00            2
3         4            4         3 2023-05-14 12:29:00           65
4         5            5         4 2023-10-15 10:40:00           83


In [255]:
#Verify whether the employee_id assignment in the previous step assign the right employee_id to the right store_id

#Create a dictionary mapping store_id to a list of employee_id
employee_id_dict = complete_employees_df.groupby('store_id')['employee_id'].apply(list).to_dict()

#check if employee_id belongs to the list of employee_ids for the corresponding store_id
def is_employee_id_correct(row):
    store_id = row['store_id']
    employee_id = row['employee_id']
    if store_id in employee_id_dict:
        return employee_id in employee_id_dict[store_id]
    else:
        return False


partial_sales['employee_id_match'] = partial_sales.apply(is_employee_id_correct, axis=1)

print(partial_sales)

      sales_id  customer_id  store_id          sales_date  employee_id  \
0            1            1         3 2023-07-14 00:17:00           33   
1            2            2         2 2023-08-03 11:28:00           28   
2            3            3         1 2023-05-11 00:50:00            2   
3            4            4         3 2023-05-14 12:29:00           65   
4            5            5         4 2023-10-15 10:40:00           83   
...        ...          ...       ...                 ...          ...   
1995      1996         1989         3 2023-12-28 19:12:00           33   
1996      1997         1990         3 2023-11-28 06:26:00           57   
1997      1998         1991         3 2023-09-08 19:57:00           47   
1998      1999         1992         1 2023-12-14 06:04:00           14   
1999      2000         1993         3 2023-09-28 13:35:00           13   

      employee_id_match  
0                  True  
1                  True  
2                  True  
3      

In [261]:
#Find rows where employee_id_match is False
false_rows = partial_sales[partial_sales['employee_id_match'] == False]

if false_rows.empty:
    print("There are no False values")
else:
    print("False rows")
    print(false_rows)


There are no False values


In [262]:
#remove the employee_id_match column
partial_sales.drop(columns=['employee_id_match'], inplace=True)

print(partial_sales.head())


   sales_id  customer_id  store_id          sales_date  employee_id
0         1            1         3 2023-07-14 00:17:00           33
1         2            2         2 2023-08-03 11:28:00           28
2         3            3         1 2023-05-11 00:50:00            2
3         4            4         3 2023-05-14 12:29:00           65
4         5            5         4 2023-10-15 10:40:00           83


In [263]:
complete_sales = partial_sales.copy()

#Now output this into a csv file for record
complete_sales.to_csv("Sales Table Insertion.csv", index = False)

print(complete_sales.head())

   sales_id  customer_id  store_id          sales_date  employee_id
0         1            1         3 2023-07-14 00:17:00           33
1         2            2         2 2023-08-03 11:28:00           28
2         3            3         1 2023-05-11 00:50:00            2
3         4            4         3 2023-05-14 12:29:00           65
4         5            5         4 2023-10-15 10:40:00           83


In [264]:
print(len(complete_sales))

2000


In [289]:
#Now we came back and work with customers_sales table

#First, create a product_id column by matching the product_id in the product_table_df

#create a dictionary mapping Product Name to product_id
product_mapping = product_table_df.set_index('product_name')['product_id'].to_dict()

#Map Product Name to product_id in the customers_sales
customers_sales['product_id'] = customers_sales['Product Name'].map(product_mapping)

print(customers_sales.head())


  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

     Date of Purchase  Product Name  Quantity  Price per Unit  sales_id  \
0 2023-07-14 00:17:00         Roses         6          155.82         1   
1 2023-07-14 00:17:00    Roast Beef        10          147.62         1   
2 2023-07-14 00:17:00  Frozen Pizza        10           26.13         1   
3 2023-07-14 00:17:00   Cough Syrup         8           53.08         1   
4 2023-07-14 00:17:00       Lettuce         8           83.83         1   

   customer_id  store_id  product_id  
0            1         3          25  
1            1         3          14  
2            1         3          20  
3       

In [291]:
#Now I need to create another column called price_paid, which is Quantity x Price per Unit
# Create the price_paid column
customers_sales['price_paid'] = customers_sales['Quantity'] * customers_sales['Price per Unit']


print(customers_sales.head())


  Customer Name     Customer Email                Store Name  \
0   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
1   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
2   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
3   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   
4   Kathy Jones  ucook@example.net  ABC Foodmart - Bay Ridge   

     Date of Purchase  Product Name  Quantity  Price per Unit  sales_id  \
0 2023-07-14 00:17:00         Roses         6          155.82         1   
1 2023-07-14 00:17:00    Roast Beef        10          147.62         1   
2 2023-07-14 00:17:00  Frozen Pizza        10           26.13         1   
3 2023-07-14 00:17:00   Cough Syrup         8           53.08         1   
4 2023-07-14 00:17:00       Lettuce         8           83.83         1   

   customer_id  store_id  product_id  price_paid  
0            1         3          25      934.92  
1            1         3          14     1476.20  
2          

In [293]:
#Now we noticed that for the same customer in the same sales_id
#There is redundant product_id in separate rows, I need to sum them together

#First, check how many duplicates are there

#Filter the DataFrame to include only duplicates based on customer_id, store_id, and sales_id
duplicates = customers_sales[customers_sales.duplicated(subset=['customer_id', 'store_id', 'sales_id', 'product_id'], keep=False)]

#Count the number of duplicates
num_duplicates = len(duplicates)

print("Number of duplicates:", num_duplicates)

Number of duplicates: 7235


In [295]:
print(len(customers_sales))
print(num_duplicates)
print(len(customers_sales) - num_duplicates/2)

22205
7235
18587.5


In [296]:
#It is better to filter out the columns needed in the SpecificSales
# Select the specific columns from customers_sales
specific_sales = customers_sales[['product_id', 'sales_id', 'Quantity', 'price_paid']].copy()

print(specific_sales.head())

   product_id  sales_id  Quantity  price_paid
0          25         1         6      934.92
1          14         1        10     1476.20
2          20         1        10      261.30
3           2         1         8      424.64
4           5         1         8      670.64


In [297]:
print(len(specific_sales))
print(len(customers_sales))


unique_sales_id_customers_sales = customers_sales['sales_id'].nunique()

unique_sales_id_specific_sales = specific_sales['sales_id'].nunique()

print(unique_sales_id_customers_sales)
print(unique_sales_id_specific_sales)

22205
22205
2000
2000


In [298]:
#Ensure that the number of duplication still remain in specific_sales for manipulation
duplicates_specific = specific_sales[specific_sales.duplicated(subset=['sales_id', 'product_id'], keep=False)]

#Count the number of duplicates
num_duplicates_specific = len(duplicates_specific)

print("Number of duplicates:", num_duplicates_specific)

Number of duplicates: 7235


In [299]:
#Now remove the duplication by group by sales_id and product_id and sum the Quantity and price_paid
specific_sales_summed = specific_sales.groupby(['sales_id', 'product_id']).agg({'Quantity': 'sum', 'price_paid': 'sum'}).reset_index()


print(specific_sales_summed.head())
print(len(specific_sales_summed))

   sales_id  product_id  Quantity  price_paid
0         1           2         8      424.64
1         1           5         8      670.64
2         1          10         7      428.12
3         1          12         1      117.04
4         1          14        10     1476.20
18333


In [302]:
#Check if all the sales_id and product_id combination in customers_sales is still being captured

#Perform a left join between customers_sales and specific_sales_summed on sales_id and product_id
merged_sales = customers_sales.merge(specific_sales_summed, on=['sales_id', 'product_id'], how='left', suffixes=('_original', '_summed'))

#Check for any null values in sales_id or product_id from the specific_sales_summed table
missing_combinations = merged_sales[merged_sales['sales_id'].isnull() | merged_sales['product_id'].isnull()]

print("Missing combinations:")
print(missing_combinations)


Missing combinations:
Empty DataFrame
Columns: [Customer Name, Customer Email, Store Name, Date of Purchase, Product Name, Quantity_original, Price per Unit, sales_id, customer_id, store_id, product_id, price_paid_original, Quantity_summed, price_paid_summed]
Index: []


In [303]:
#Since all the combinations are captured, we could export this specific_sales_summed into csv
#change the column name Quantity to quantity
specific_sales_summed = specific_sales_summed.rename(columns={'Quantity': 'quantity'})

specific_sales_final = specific_sales_summed.copy()

print(len(specific_sales_final))

18333


In [304]:
#Read into a csv file
specific_sales_final.to_csv("SpecificSales Table Insertion.csv", index = False)

In [306]:
#Now we need to work on the Payment Table
#This table first require two variable from the specific_sales_final table
#sales_id from the table
#sum of the price_paid per sales_id
payment_table = specific_sales_final.groupby('sales_id')['price_paid'].sum().reset_index()
print(payment_table.head())
print(len(payment_table))

   sales_id  price_paid
0         1     6364.10
1         2     4413.43
2         3     7399.43
3         4     5561.65
4         5     2705.85
2000


In [307]:
#The result confirm the number of transactions from customers_sales table
#Now we need to generate other columns and values based on the current payment_table

#Define payment methods
payment_methods = ["Cash", "Check", "Credit Card", "Debit Card", "PayPal", "Venmo", "Bank Transfer"]

sales_ids = payment_table['sales_id'].unique()

#Generate random payment methods for each sales_id
random_payment_methods = np.random.choice(payment_methods, size=len(sales_ids))

#Create a DataFrame for payment methods
payment_method_df = pd.DataFrame({
    'sales_id': sales_ids,
    'payment_method': random_payment_methods
})

# Merge the payment_method_df with the payment_table
payment_table = pd.merge(payment_table, payment_method_df, on='sales_id', how='left')

In [308]:
print(payment_table.head())
print(len(payment_table))

   sales_id  price_paid payment_method
0         1     6364.10          Check
1         2     4413.43  Bank Transfer
2         3     7399.43           Cash
3         4     5561.65         PayPal
4         5     2705.85           Cash
2000


In [309]:
#Assign payment_id to each row
payment_table['payment_id'] = range(1, len(payment_table) + 1)
print(payment_table.head())

   sales_id  price_paid payment_method  payment_id
0         1     6364.10          Check           1
1         2     4413.43  Bank Transfer           2
2         3     7399.43           Cash           3
3         4     5561.65         PayPal           4
4         5     2705.85           Cash           5


In [312]:
#Provide discount percentage to each row

#Get the number of payments
num_payments = len(payment_table)

#Generate random discount values
discount_values = np.random.randint(0, 41, num_payments)


payment_table['discount'] = discount_values

print(payment_table.head())

   sales_id  price_paid payment_method  payment_id  discount
0         1     6364.10          Check           1        19
1         2     4413.43  Bank Transfer           2        11
2         3     7399.43           Cash           3        34
3         4     5561.65         PayPal           4        21
4         5     2705.85           Cash           5        16


In [313]:
#To calculate the tax rate, we set the tax rate at 8.76%, which is 0.0875

#Define the tax rate
tax_rate = 0.0875  # 8.75%

#Calculate the tax amount
payment_table['tax_amount'] = payment_table['price_paid'] * tax_rate

print(payment_table.head())

   sales_id  price_paid payment_method  payment_id  discount  tax_amount
0         1     6364.10          Check           1        19  556.858750
1         2     4413.43  Bank Transfer           2        11  386.175125
2         3     7399.43           Cash           3        34  647.450125
3         4     5561.65         PayPal           4        21  486.644375
4         5     2705.85           Cash           5        16  236.761875


In [314]:
#Now calculate the total_payment value by summing above values together

#Calculate the total payment
payment_table['total_payment'] = payment_table['price_paid'] * (1 - (payment_table['discount'] / 100)) + payment_table['tax_amount']

# Round the total payment to two decimal places
payment_table['total_payment'] = payment_table['total_payment'].round(2)

print(payment_table.head(10))


   sales_id  price_paid payment_method  payment_id  discount  tax_amount  \
0         1     6364.10          Check           1        19  556.858750   
1         2     4413.43  Bank Transfer           2        11  386.175125   
2         3     7399.43           Cash           3        34  647.450125   
3         4     5561.65         PayPal           4        21  486.644375   
4         5     2705.85           Cash           5        16  236.761875   
5         6     5686.00          Venmo           6         1  497.525000   
6         7     2671.01         PayPal           7         6  233.713375   
7         8     9174.85     Debit Card           8        18  802.799375   
8         9     5061.40         PayPal           9        27  442.872500   
9        10    10906.79  Bank Transfer          10        14  954.344125   

   total_payment  
0        5711.78  
1        4314.13  
2        5531.07  
3        4880.35  
4        2509.68  
5        6126.66  
6        2744.46  
7        83

In [315]:
#Round the tax_amount as well
payment_table['tax_amount'] = payment_table['tax_amount'].round(2)

print(payment_table.head(10))

   sales_id  price_paid payment_method  payment_id  discount  tax_amount  \
0         1     6364.10          Check           1        19      556.86   
1         2     4413.43  Bank Transfer           2        11      386.18   
2         3     7399.43           Cash           3        34      647.45   
3         4     5561.65         PayPal           4        21      486.64   
4         5     2705.85           Cash           5        16      236.76   
5         6     5686.00          Venmo           6         1      497.52   
6         7     2671.01         PayPal           7         6      233.71   
7         8     9174.85     Debit Card           8        18      802.80   
8         9     5061.40         PayPal           9        27      442.87   
9        10    10906.79  Bank Transfer          10        14      954.34   

   total_payment  
0        5711.78  
1        4314.13  
2        5531.07  
3        4880.35  
4        2509.68  
5        6126.66  
6        2744.46  
7        83

In [316]:
#Now drop the price_paid column for generating the Payment Table
payment_table.drop(columns=['price_paid'], inplace=True)

print(payment_table.head())
print(len(payment_table))

   sales_id payment_method  payment_id  discount  tax_amount  total_payment
0         1          Check           1        19      556.86        5711.78
1         2  Bank Transfer           2        11      386.18        4314.13
2         3           Cash           3        34      647.45        5531.07
3         4         PayPal           4        21      486.64        4880.35
4         5           Cash           5        16      236.76        2509.68
2000


In [317]:
#Now since the table is ready, export the table into a csv file for record
payment_table.to_csv("Payment Table Insertion.csv", index = False)

### Generate the dataframe and csv file for these 8 table insertion
- Sales Table -- complete_sales (already generated from previous steps)
- Specific Sales Table -- specific_sales_final
- Payment Table -- payment_table
- Customer Table -- df_customer_selected (already there from previous step)
- LoyaltyProgram Table -- loyalty_program (already there from previous step)
- Employee Table -- complete_employees_df (already there from previous step)
- CustomerRating Table -- complete_review (already there from previous step)
- OperatingCost Table -- operatingcost_df (already there from previous step)


- Category Table -- category_df
- Product Price Table -- final_product_price_df
- Product Table -- product_table_df
- product_orders Table -- product_orders_table_df
- Supplier Table -- supplier_table_df
- Orders Table -- orders_table_df
- store_orders Table -- stores_orders_table_df
- Store Table -- store_table_df


## All Data are manipulated for data insertion
- Sales Table -- complete_sales 
- Specific Sales Table -- specific_sales_final
- Payment Table -- payment_table
- Customer Table -- df_customer_selected 
- LoyaltyProgram Table -- loyalty_program 
- Employee Table -- complete_employees_df 
- CustomerRating Table -- complete_review 
- OperatingCost Table -- operatingcost_df 


- Category Table -- category_df
- Product Price Table -- final_product_price_df
- Product Table -- product_table_df
- product_orders Table -- product_orders_table_df
- Supplier Table -- supplier_table_df
- Orders Table -- orders_table_df
- store_orders Table -- stores_orders_table_df
- Store Table -- store_table_df

## Insert Data into Database Schema based on Schema Creation sequence

The Seqence and the related table that is used for data insertion
- Category Table -- category_df
- Product Table -- product_table_df
- Product Price Table -- final_product_price_df
- Supplier Table -- supplier_table_df
- Store Table -- store_table_df
- LoyaltyProgram Table -- loyalty_program 
- Customer Table -- df_customer_selected 
- Employee Table -- complete_employees_df 
- Orders Table -- orders_table_df
- Sales Table -- complete_sales 
- store_orders Table -- stores_orders_table_df
- product_orders Table -- product_orders_table_df
- Specific Sales Table -- specific_sales_final
- Payment Table -- payment_table
- CustomerRating Table -- complete_review 
- OperatingCost Table -- operatingcost_df 


### Insert Data to Category Table
Two method available:
- direct insert using the dataframe generated: category_df
- read in the csv file generated from previous step

In [320]:
print(category_df.head())
print(len(category_df))

   category_id  category_name
0            1      beverages
1            2       pharmacy
2            3          meats
3            4         floral
4            5  fresh produce
11


In [456]:
#Direct insert using the dataframe generated: category_df
#There should be 11 rows in the SQL database
#The dataframe variables is already in the right sequence

#Transform the category_df into a list of tuples
category_insert = list(category_df.itertuples(index=False, name=None))

for category in category_insert:
     cur.execute("INSERT INTO Category (category_id, category_name) VALUES (%s, %s);", category)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method
#category_df = pd.read_csv("Category Table Insertion.csv")
#Transform the category_df into a list of tuples
#category_insert = list(category_df.itertuples(index=False, name=None))

#for category in category_insert:
     #cur.execute("INSERT INTO Category (category_id, category_name) VALUES (%s, %s);", category)

#conn.commit()

In [457]:
#check the table and the values
cur.execute("SELECT * FROM Category;")
rows_category = cur.fetchall()
for value in rows_category:
    print(value)

print(len(rows_category))

(1, 'beverages')
(2, 'pharmacy')
(3, 'meats')
(4, 'floral')
(5, 'fresh produce')
(6, 'dairy')
(7, 'seafood')
(8, 'frozen foods')
(9, 'bakery')
(10, 'deli')
(11, 'dry goods')
11


### Insert Data to Product Table
Two method available:
- direct insert using the dataframe generated: product_table_df
- read in the csv file generated from previous step

In [321]:
print(product_table_df.head())
print(len(product_table_df))


   product_id product_name  category_id  \
0           1        Water            1   
1           2  Cough Syrup            2   
2           3   Beef Steak            3   
3           4       Tulips            4   
4           5      Lettuce            5   

                                 product_description  
0  High-quality beverages item, Water, sourced re...  
1  High-quality pharmacy item, Cough Syrup, sourc...  
2  High-quality meats item, Beef Steak, sourced r...  
3  High-quality floral item, Tulips, sourced resp...  
4  High-quality fresh produce item, Lettuce, sour...  
33


In [458]:
#Direct insert using the dataframe generated: product_table_df
#There should be 33 rows in the SQL database
#The dataframe variables is already in the right sequence

#Transform the product_table_df into a list of tuples
product_insert = [tuple(row) for row in product_table_df.values]

for product in product_insert:
    cur.execute("INSERT INTO Product (product_id, product_name, category_id, product_description) VALUES (%s, %s, %s, %s);", product)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#product_table_df = pd.read_csv("Product Table Insertion.csv")

#Transform the product_table_df into a list of tuples
#product_insert = [tuple(row) for row in product_table_df.values]

#for product in product_insert:
    #cur.execute("INSERT INTO Product (product_id, product_name, category_id, product_description) VALUES (%s, %s, %s, %s);", product)

#conn.commit()


In [459]:
#check the table and the values
cur.execute("SELECT * FROM Product;")
rows_product = cur.fetchall()
for value in rows_product:
    print(value)

print(len(rows_product))

(1, 'Water', 1, 'High-quality beverages item, Water, sourced responsibly.')
(2, 'Cough Syrup', 2, 'High-quality pharmacy item, Cough Syrup, sourced responsibly.')
(3, 'Beef Steak', 3, 'High-quality meats item, Beef Steak, sourced responsibly.')
(4, 'Tulips', 4, 'High-quality floral item, Tulips, sourced responsibly.')
(5, 'Lettuce', 5, 'High-quality fresh produce item, Lettuce, sourced responsibly.')
(6, 'Cheese', 6, 'High-quality dairy item, Cheese, sourced responsibly.')
(7, 'Band-Aids', 2, 'High-quality pharmacy item, Band-Aids, sourced responsibly.')
(8, 'Shrimp', 7, 'High-quality seafood item, Shrimp, sourced responsibly.')
(9, 'Carrots', 5, 'High-quality fresh produce item, Carrots, sourced responsibly.')
(10, 'Frozen Vegetables', 8, 'High-quality frozen foods item, Frozen Vegetables, sourced responsibly.')
(11, 'Chicken Breast', 3, 'High-quality meats item, Chicken Breast, sourced responsibly.')
(12, 'Juice', 1, 'High-quality beverages item, Juice, sourced responsibly.')
(13, 'B

### Insert Data to ProductPrice Table
Two method available:
- direct insert using the dataframe generated: final_product_price_df
- read in the csv file generated from previous step

In [325]:
print(final_product_price_df.head())
print(len(final_product_price_df))

   product_id price_date  product_price
0           1 2024-04-20       37.14100
1           1 2024-04-21       87.03825
2           1 2024-04-26      113.93200
3           1 2024-05-04       46.69600
4           1 2024-05-06       78.74100
3895


In [342]:
#convert final_product_price_df product_price to two decimal place
final_product_price_df['product_price'] = final_product_price_df['product_price'].round(2)

print(final_product_price_df.head())

   product_id price_date  product_price
0           1 2024-04-20          37.14
1           1 2024-04-21          87.04
2           1 2024-04-26         113.93
3           1 2024-05-04          46.70
4           1 2024-05-06          78.74


In [460]:
#Direct insert using the dataframe generated: final_product_price_df
#There should be 3895 rows in the SQL database
#The dataframe variables is already in the right sequence
#Convert the price_date column to datetime format to make sure the format apply DATE variable type
final_product_price_df['price_date'] = pd.to_datetime(final_product_price_df['price_date'])

productprice_insert = [tuple(row) for row in final_product_price_df.values]

for productprice in productprice_insert:
    cur.execute("INSERT INTO ProductPrice (product_id, price_date, product_price) VALUES (%s, %s, %s);", productprice)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#final_product_price_df = pd.read_csv("ProductPrice Table Insertion.csv")

#final_product_price_df['price_date'] = pd.to_datetime(final_product_price_df['price_date'])

#convert final_product_price_df product_price to two decimal place
#final_product_price_df['product_price'] = final_product_price_df['product_price'].round(2)

#print(final_product_price_df.head())

#productprice_insert = [tuple(row) for row in final_product_price_df.values]

#for productprice in productprice_insert:
    #cur.execute("INSERT INTO ProductPrice (product_id, price_date, product_price) VALUES (%s, %s, %s);", productprice)

#conn.commit()

In [461]:
#check the table and the values
cur.execute("SELECT * FROM ProductPrice;")
rows_productprice = cur.fetchall()
for value in rows_productprice:
    print(value)

print(len(rows_productprice))

(1, datetime.date(2024, 4, 20), Decimal('37.14'))
(1, datetime.date(2024, 4, 21), Decimal('87.04'))
(1, datetime.date(2024, 4, 26), Decimal('113.93'))
(1, datetime.date(2024, 5, 4), Decimal('46.70'))
(1, datetime.date(2024, 5, 6), Decimal('78.74'))
(1, datetime.date(2024, 5, 19), Decimal('129.48'))
(1, datetime.date(2024, 5, 21), Decimal('28.39'))
(1, datetime.date(2024, 5, 23), Decimal('65.40'))
(1, datetime.date(2024, 5, 24), Decimal('97.28'))
(1, datetime.date(2024, 5, 27), Decimal('125.12'))
(1, datetime.date(2024, 6, 9), Decimal('64.95'))
(1, datetime.date(2024, 6, 10), Decimal('3.95'))
(1, datetime.date(2024, 6, 19), Decimal('40.42'))
(1, datetime.date(2024, 6, 22), Decimal('19.77'))
(1, datetime.date(2024, 6, 23), Decimal('56.69'))
(1, datetime.date(2024, 6, 27), Decimal('44.52'))
(1, datetime.date(2024, 6, 30), Decimal('40.20'))
(1, datetime.date(2024, 7, 3), Decimal('105.00'))
(1, datetime.date(2024, 7, 8), Decimal('85.92'))
(1, datetime.date(2024, 7, 9), Decimal('44.37'))
(1,

### Insert Data to Supplier Table
Two method available:
- direct insert using the dataframe generated: supplier_table_df
- read in the csv file generated from previous step

In [326]:

print(supplier_table_df.head())
print(len(supplier_table_df))

   supplier_id                          name                       email
0            1  Wong, Mcdonald and Rodriguez  elizabethdavis@example.com
1            2               Hamilton-Burton     scottedward@example.net
2            3                  Cardenas Ltd     robinmoreno@example.com
3            4                  Matthews LLC  guerreroconnie@example.net
4            5   Hamilton, Briggs and Nguyen     herringeric@example.org
50


In [462]:
#Direct insert using the dataframe generated: supplier_table_df
#There should be 50 rows in the SQL database
#The dataframe variables is already in the right sequence
supplier_insert = [tuple(row) for row in supplier_table_df.values]

for supplier in supplier_insert:
    cur.execute("INSERT INTO Supplier (supplier_id, name, email) VALUES (%s, %s, %s);", supplier)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#supplier_table_df = pd.read_csv("Supplier Table Insertion.csv")

#supplier_insert = [tuple(row) for row in supplier_table_df.values]

#for supplier in supplier_insert:
    #cur.execute("INSERT INTO Supplier (supplier_id, name, email) VALUES (%s, %s, %s);", supplier)

#conn.commit()

In [463]:
#check the table and the values
cur.execute("SELECT * FROM Supplier;")
rows_supplier = cur.fetchall()
for value in rows_supplier:
    print(value)

print(len(rows_supplier))

(1, 'Wong, Mcdonald and Rodriguez', 'elizabethdavis@example.com')
(2, 'Hamilton-Burton', 'scottedward@example.net')
(3, 'Cardenas Ltd', 'robinmoreno@example.com')
(4, 'Matthews LLC', 'guerreroconnie@example.net')
(5, 'Hamilton, Briggs and Nguyen', 'herringeric@example.org')
(6, 'Lowery-Abbott', 'darrylwilliams@example.net')
(7, 'Silva PLC', 'ohowe@example.org')
(8, 'Patterson and Sons', 'andrea86@example.com')
(9, 'Wallace, Brown and Garcia', 'moorepaul@example.net')
(10, 'West, Jones and Hatfield', 'natasha72@example.com')
(11, 'Giles-Greene', 'shawn47@example.org')
(12, 'Crawford, Oliver and Anderson', 'dawndaniel@example.net')
(13, 'Garcia, David and Mendoza', 'phillipcombs@example.net')
(14, 'Neal-Jones', 'rodney30@example.net')
(15, 'Molina-Rivera', 'holly61@example.net')
(16, 'Gonzalez-Diaz', 'jacksonchristopher@example.com')
(17, 'Gray, Roman and Little', 'rachel74@example.com')
(18, 'Watts-Brown', 'rgiles@example.net')
(19, 'Wang-Combs', 'jefferygalvan@example.org')
(20, 'Cole 

### Insert Data to Store Table
Two method available:
- direct insert using the dataframe generated: store_table_df
- read in the csv file generated from previous step

In [327]:
print(store_table_df.head())
print(len(store_table_df))

   store_id                    store_name contact_number  \
0         1          ABC Foodmart - DUMBO     3471231234   
1         2        ABC Foodmart - Tribeca     3479879876   
2         3      ABC Foodmart - Bay Ridge     3474564565   
3         4     ABC Foodmart - Whitestone     3478768765   
4         5  ABC Foodmart - Staten Island     3471234567   

                     street           city state  
0            123 Jay Street       Brooklyn    NY  
1       133 Chambers Street       New York    NY  
2              8518 3rd Ave       Brooklyn    NY  
3  153-65 Cross Island Pkwy     Whitestone    NY  
4         2655 Richmond Ave  Staten Island    NY  
5


In [464]:
#Direct insert using the dataframe generated: store_table_df
#There should be 5 rows in the SQL database
#The dataframe variables is already in the right sequence
store_insert = [tuple(row) for row in store_table_df.values]

for store in store_insert:
    cur.execute("INSERT INTO Store (store_id, store_name, contact_number, street, city, state) VALUES (%s, %s, %s, %s, %s, %s);", store)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#store_table_df = pd.read_csv("Store Table Insertion.csv")

#store_insert = [tuple(row) for row in store_table_df.values]

#for store in store_insert:
    #cur.execute("INSERT INTO Store (store_id, store_name, contact_number, street, city, state) VALUES (%s, %s, %s, %s, %s, %s);", store)

#conn.commit()

In [465]:
#check the table and the values
cur.execute("SELECT * FROM Store;")
rows_store = cur.fetchall()
for value in rows_store:
    print(value)

print(len(rows_store))

(1, 'ABC Foodmart - DUMBO', '3471231234', '123 Jay Street', 'Brooklyn', 'NY')
(2, 'ABC Foodmart - Tribeca', '3479879876', '133 Chambers Street', 'New York', 'NY')
(3, 'ABC Foodmart - Bay Ridge', '3474564565', '8518 3rd Ave', 'Brooklyn', 'NY')
(4, 'ABC Foodmart - Whitestone', '3478768765', '153-65 Cross Island Pkwy', 'Whitestone', 'NY')
(5, 'ABC Foodmart - Staten Island', '3471234567', '2655 Richmond Ave', 'Staten Island', 'NY')
5


### Insert Data to LoyaltyProgram Table
Two method available:
- direct insert using the dataframe generated: loyalty_program
- read in the csv file generated from previous step

In [328]:
print(loyalty_program.head())
print(len(loyalty_program))

   loyalty_id     level  points
0           1    silver    5958
1           2    silver    8001
2           3    silver    9601
3           4      gold    4926
4           5  platinum    3644
1993


In [466]:
#Direct insert using the dataframe generated: loyalty_program
#The dataframe variables is already in the right sequence
loyalty_insert = [tuple(row) for row in loyalty_program.values]

for loyalty in loyalty_insert:
    cur.execute("INSERT INTO LoyaltyProgram (loyalty_id, level, points) VALUES (%s, %s, %s);", loyalty)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#loyalty_program = pd.read_csv("LoyaltyProgram Table Insertion.csv")

#loyalty_insert = [tuple(row) for row in loyalty_program.values]

#for loyalty in loyalty_insert:
    #cur.execute("INSERT INTO LoyaltyProgram (loyalty_id, level, points) VALUES (%s, %s, %s);", loyalty)

#conn.commit()

In [467]:
#check the table and the values
cur.execute("SELECT * FROM LoyaltyProgram;")
rows_loyalty = cur.fetchall()
for value in rows_loyalty:
    print(value)

print(len(rows_loyalty))

(1, 'silver', 5958)
(2, 'silver', 8001)
(3, 'silver', 9601)
(4, 'gold', 4926)
(5, 'platinum', 3644)
(6, 'platinum', 1370)
(7, 'silver', 2858)
(8, 'silver', 7132)
(9, 'gold', 4271)
(10, 'gold', 4180)
(11, 'silver', 8634)
(12, 'silver', 9451)
(13, 'silver', 427)
(14, 'gold', 4997)
(15, 'gold', 9125)
(16, 'platinum', 9732)
(17, 'silver', 356)
(18, 'platinum', 8456)
(19, 'platinum', 9459)
(20, 'gold', 3082)
(21, 'gold', 5224)
(22, 'silver', 9314)
(23, 'gold', 2660)
(24, 'silver', 6275)
(25, 'gold', 3903)
(26, 'gold', 6119)
(27, 'gold', 5661)
(28, 'silver', 4742)
(29, 'gold', 962)
(30, 'gold', 1099)
(31, 'silver', 7168)
(32, 'gold', 8998)
(33, 'silver', 8689)
(34, 'platinum', 3357)
(35, 'platinum', 5962)
(36, 'platinum', 2668)
(37, 'silver', 9908)
(38, 'platinum', 7944)
(39, 'platinum', 9895)
(40, 'silver', 7527)
(41, 'platinum', 7229)
(42, 'silver', 1083)
(43, 'silver', 6225)
(44, 'silver', 2082)
(45, 'gold', 4104)
(46, 'silver', 3147)
(47, 'platinum', 5655)
(48, 'silver', 6210)
(49, 'silv

### Insert Data to Customer Table
Two method available:
- direct insert using the dataframe generated: df_customer_selected
- read in the csv file generated from previous step

In [329]:
print(df_customer_selected.head())
print(len(df_customer_selected))

    customer_id first_name   last_name       phone                      email  \
0             1      Kathy       Jones  8243343078          ucook@example.net   
13            2    Richard        Diaz  2578799891    stephanie93@example.com   
25            3     Steven      Medina  3932152834       dsanchez@example.net   
38            4     Angela  Washington  6794472057  vickimitchell@example.com   
47            5    Valerie    Crawford  3886602382    sabrinawade@example.org   

    loyalty_id  
0            1  
13           2  
25           3  
38           4  
47           5  
1993


In [330]:
#change the sequence of the columns first
# Reorder the columns
df_customer_selected = df_customer_selected[['customer_id', 'first_name', 'last_name', 'email', 'phone', 'loyalty_id']]
print(df_customer_selected.head())

    customer_id first_name   last_name                      email       phone  \
0             1      Kathy       Jones          ucook@example.net  8243343078   
13            2    Richard        Diaz    stephanie93@example.com  2578799891   
25            3     Steven      Medina       dsanchez@example.net  3932152834   
38            4     Angela  Washington  vickimitchell@example.com  6794472057   
47            5    Valerie    Crawford    sabrinawade@example.org  3886602382   

    loyalty_id  
0            1  
13           2  
25           3  
38           4  
47           5  


In [468]:
#Direct insert using the dataframe generated: df_customer_selected
#The dataframe variables is already in the right sequence
customer_insert = [tuple(row) for row in df_customer_selected.values]

for customer in customer_insert:
    cur.execute("INSERT INTO Customer (customer_id, first_name, last_name, email, phone, loyalty_id) VALUES (%s, %s, %s, %s, %s, %s);", customer)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#df_customer_selected = pd.read_csv("Customer Table Insertion.csv")

#df_customer_selected = df_customer_selected[['customer_id', 'first_name', 'last_name', 'email', 'phone', 'loyalty_id']]
#print(df_customer_selected.head())

#customer_insert = [tuple(row) for row in df_customer_selected.values]

#for customer in customer_insert:
    #cur.execute("INSERT INTO Customer (customer_id, first_name, last_name, email, phone, loyalty_id) VALUES (%s, %s, %s, %s, %s, %s);", customer)

#conn.commit()

In [469]:
#check the table and the values
cur.execute("SELECT * FROM Customer;")
rows_customer = cur.fetchall()
for value in rows_customer:
    print(value)

print(len(rows_customer))

(1, 'Kathy', 'Jones', 'ucook@example.net', '8243343078', 1)
(2, 'Richard', 'Diaz', 'stephanie93@example.com', '2578799891', 2)
(3, 'Steven', 'Medina', 'dsanchez@example.net', '3932152834', 3)
(4, 'Angela', 'Washington', 'vickimitchell@example.com', '6794472057', 4)
(5, 'Valerie', 'Crawford', 'sabrinawade@example.org', '3886602382', 5)
(6, 'Heather', 'Harris', 'colleenbanks@example.net', '8016283163', 6)
(7, 'Michael', 'Tate', 'barbara17@example.net', '5483236415', 7)
(8, 'Heidi', 'Garcia', 'mayvalerie@example.com', '8909611558', 8)
(9, 'Maria', 'Martinez', 'cathymunoz@example.com', '7998220226', 9)
(10, 'Manuel', 'Mcdowell', 'bendermelissa@example.net', '2367791606', 10)
(11, 'Michael', 'Morris', 'sandraarnold@example.org', '8789807518', 11)
(12, 'Brittany', 'Newman', 'weberkelly@example.net', '1971010428', 12)
(13, 'Bobby', 'Johnson', 'christopher72@example.net', '1550843953', 13)
(14, 'Derek', 'Johnson', 'jpeck@example.com', '2540252343', 14)
(15, 'Justin', 'Moore', 'michelle28@examp

### Insert Data to Employee Table
Two method available:
- direct insert using the dataframe generated: complete_employees_df
- read in the csv file generated from previous step

In [331]:
print(complete_employees_df.head())
print(len(complete_employees_df))

      employee_id first_name last_name          position  store_id  salary
4944            9        Jay   Walters           cashier         1   18.98
4975           20     Monica    Martin            bakery         3   15.59
4820           32     Thomas      Love            floral         5   12.72
4902           78     Ashley    Horton          pharmacy         3   16.58
4994           37     Joshua    Church  customer service         3   12.93
100


In [470]:
#Direct insert using the dataframe generated: complete_employees_df
#There should be 100 rows in the SQL database
#The dataframe variables is already in the right sequence
employee_insert = [tuple(row) for row in complete_employees_df.values]

for employee in employee_insert:
    cur.execute("INSERT INTO Employee (employee_id, first_name, last_name, position, store_id, salary) VALUES (%s, %s, %s, %s, %s, %s);", employee)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#complete_employees_df = pd.read_csv("Employee Table Insertion.csv")

#employee_insert = [tuple(row) for row in complete_employees_df.values]

#for employee in employee_insert:
    #cur.execute("INSERT INTO Employee (employee_id, first_name, last_name, position, store_id, salary) VALUES (%s, %s, %s, %s, %s, %s);", employee)

#conn.commit()

In [471]:
#check the table and the values
cur.execute("SELECT * FROM Employee;")
rows_employee = cur.fetchall()
for value in rows_employee:
    print(value)

print(len(rows_employee))

(9, 'Jay', 'Walters', 'cashier', 1, Decimal('18.98'))
(20, 'Monica', 'Martin', 'bakery', 3, Decimal('15.59'))
(32, 'Thomas', 'Love', 'floral', 5, Decimal('12.72'))
(78, 'Ashley', 'Horton', 'pharmacy', 3, Decimal('16.58'))
(37, 'Joshua', 'Church', 'customer service', 3, Decimal('12.93'))
(53, 'Brandon', 'Nichols', 'bakery', 1, Decimal('18.63'))
(82, 'Brandy', 'Smith', 'fresh produce', 4, Decimal('18.36'))
(31, 'Evelyn', 'Jones', 'cashier', 2, Decimal('13.71'))
(97, 'Kirsten', 'Long', 'floral', 5, Decimal('15.77'))
(44, 'Jennifer', 'Davis', 'cashier', 5, Decimal('13.54'))
(68, 'Frederick', 'Conrad', 'dairy', 5, Decimal('16.73'))
(79, 'Brenda', 'Alvarez', 'pharmacy', 2, Decimal('19.05'))
(38, 'Marcus', 'Woods', 'floral', 1, Decimal('15.30'))
(11, 'Monica', 'Wells', 'pharmacy', 5, Decimal('19.23'))
(80, 'Samantha', 'Payne', 'frozen foods', 3, Decimal('17.15'))
(61, 'Christine', 'Roach', 'pharmacy', 2, Decimal('13.32'))
(17, 'Kathleen', 'Chen', 'fresh produce', 4, Decimal('16.76'))
(3, 'Ama

### Insert Data to Orders Table
Two method available:
- direct insert using the dataframe generated: orders_table_df
- read in the csv file generated from previous step

In [332]:
print(orders_table_df.head())
print(len(orders_table_df))

                               order_id  supplier_id stock_date
0  140ead02-1500-4660-897e-8773e7c34a6f            1 2024-06-09
1  849766ff-6b13-4de9-8169-e3cfb6832bbe            2 2025-01-09
2  4f83ccd2-5dad-4420-89c2-8365a60846d4            3 2024-06-22
3  1fcd017d-8cdb-4901-b874-39e43b3bbb0b            4 2025-02-13
4  a088b206-5765-4a0d-b384-1ed9dc274246            5 2025-02-25
1000


In [333]:
#make sure the stock_date is converted to the right format
#Convert the stock_date column to datetime format
orders_table_df['stock_date'] = pd.to_datetime(orders_table_df['stock_date'])

In [472]:
#Direct insert using the dataframe generated: orders_table_df
#There should be 1000 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
orders_insert = [tuple(row) for row in orders_table_df.values]

for order in orders_insert:
    cur.execute("INSERT INTO Orders (order_id, supplier_id, stock_date) VALUES (%s, %s, %s);", order)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#orders_table_df = pd.read_csv("Orders Table Insertion.csv")

#orders_table_df['stock_date'] = pd.to_datetime(orders_table_df['stock_date'])

#orders_insert = [tuple(row) for row in orders_table_df.values]

#for order in orders_insert:
    #cur.execute("INSERT INTO Orders (order_id, supplier_id, stock_date) VALUES (%s, %s, %s);", order)

#conn.commit()


In [473]:
#check the table and the values
cur.execute("SELECT * FROM Orders;")
rows_orders = cur.fetchall()
for value in rows_orders:
    print(value)

print(len(rows_orders))

('140ead02-1500-4660-897e-8773e7c34a6f', 1, datetime.date(2024, 6, 9))
('849766ff-6b13-4de9-8169-e3cfb6832bbe', 2, datetime.date(2025, 1, 9))
('4f83ccd2-5dad-4420-89c2-8365a60846d4', 3, datetime.date(2024, 6, 22))
('1fcd017d-8cdb-4901-b874-39e43b3bbb0b', 4, datetime.date(2025, 2, 13))
('a088b206-5765-4a0d-b384-1ed9dc274246', 5, datetime.date(2025, 2, 25))
('0cbbbc5b-b146-4449-83e2-f7e3b2adcbd7', 6, datetime.date(2025, 4, 2))
('cd963182-f3de-464d-87a1-6cbaa08a6be5', 7, datetime.date(2024, 9, 23))
('f6c6e797-4799-4dda-aca1-1e39377a6ba2', 8, datetime.date(2025, 3, 17))
('85911ba6-8f8d-4336-bbbc-b9ff3181d608', 9, datetime.date(2025, 3, 5))
('53c95ac8-d16a-46b1-a66b-a0d52c4c34ad', 10, datetime.date(2024, 12, 8))
('76819cd7-29f5-4432-9353-4db841bd3b4a', 8, datetime.date(2024, 10, 15))
('7d643604-9f8a-45a0-935f-1a2f4bd49a7d', 11, datetime.date(2025, 1, 23))
('97fcf51d-e661-4847-9e8b-c56c18d1e28d', 12, datetime.date(2024, 11, 19))
('f5050e82-9242-4e06-8e66-182429448018', 6, datetime.date(2025,

### Insert Data to Sales Table
Two method available:
- direct insert using the dataframe generated: complete_sales
- read in the csv file generated from previous step

In [334]:
print(complete_sales.head())
print(len(complete_sales))

   sales_id  customer_id  store_id          sales_date  employee_id
0         1            1         3 2023-07-14 00:17:00           33
1         2            2         2 2023-08-03 11:28:00           28
2         3            3         1 2023-05-11 00:50:00            2
3         4            4         3 2023-05-14 12:29:00           65
4         5            5         4 2023-10-15 10:40:00           83
2000


In [335]:
#Reorder the columns
complete_sales = complete_sales[['sales_id', 'customer_id', 'employee_id', 'store_id', 'sales_date']]

#Convert the sales_date format to the right format
complete_sales['sales_date'] = pd.to_datetime(complete_sales['sales_date'])

print(complete_sales.head())


   sales_id  customer_id  employee_id  store_id          sales_date
0         1            1           33         3 2023-07-14 00:17:00
1         2            2           28         2 2023-08-03 11:28:00
2         3            3            2         1 2023-05-11 00:50:00
3         4            4           65         3 2023-05-14 12:29:00
4         5            5           83         4 2023-10-15 10:40:00


In [474]:
#Direct insert using the dataframe generated: complete_sales
#There should be 2000 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
sales_insert = [tuple(row) for row in complete_sales.values]

for sale in sales_insert:
    cur.execute("INSERT INTO Sales (sales_id, customer_id, employee_id, store_id, sales_date) VALUES (%s, %s, %s, %s, %s);", sale)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#complete_sales = pd.read_csv("Sales Table Insertion.csv")

#Reorder the columns
#complete_sales = complete_sales[['sales_id', 'customer_id', 'employee_id', 'store_id', 'sales_date']]

#Convert the sales_date format to the right format
#complete_sales['sales_date'] = pd.to_datetime(complete_sales['sales_date'])

#print(complete_sales.head())

#sales_insert = [tuple(row) for row in complete_sales.values]

#for sale in sales_insert:
    #cur.execute("INSERT INTO Sales (sales_id, customer_id, employee_id, store_id, sales_date) VALUES (%s, %s, %s, %s, %s);", sale)

#conn.commit()



In [475]:
#check the table and the values
cur.execute("SELECT * FROM Sales;")
rows_sales = cur.fetchall()
for value in rows_sales:
    print(value)

print(len(rows_sales))

(1, 1, 33, 3, datetime.date(2023, 7, 14))
(2, 2, 28, 2, datetime.date(2023, 8, 3))
(3, 3, 2, 1, datetime.date(2023, 5, 11))
(4, 4, 65, 3, datetime.date(2023, 5, 14))
(5, 5, 83, 4, datetime.date(2023, 10, 15))
(6, 6, 32, 5, datetime.date(2024, 1, 1))
(7, 7, 81, 3, datetime.date(2023, 4, 22))
(8, 8, 92, 4, datetime.date(2024, 3, 31))
(9, 9, 36, 1, datetime.date(2024, 2, 15))
(10, 10, 20, 3, datetime.date(2023, 5, 29))
(11, 11, 67, 1, datetime.date(2023, 9, 5))
(12, 12, 47, 3, datetime.date(2023, 10, 14))
(13, 13, 27, 5, datetime.date(2024, 1, 8))
(14, 14, 90, 1, datetime.date(2023, 4, 23))
(15, 15, 29, 4, datetime.date(2023, 4, 19))
(16, 16, 30, 5, datetime.date(2023, 8, 31))
(17, 17, 28, 2, datetime.date(2023, 11, 27))
(18, 18, 77, 1, datetime.date(2023, 4, 15))
(19, 19, 35, 4, datetime.date(2023, 6, 9))
(20, 20, 58, 1, datetime.date(2023, 12, 28))
(21, 21, 61, 2, datetime.date(2023, 10, 22))
(22, 22, 92, 4, datetime.date(2023, 12, 3))
(23, 23, 9, 1, datetime.date(2023, 6, 15))
(24, 24,

### Insert Data to store_orders Table
Two method available:
- direct insert using the dataframe generated: stores_orders_table_df
- read in the csv file generated from previous step

In [336]:
print(stores_orders_table_df.head())
print(len(stores_orders_table_df))

   store_id                              order_id
0         5  140ead02-1500-4660-897e-8773e7c34a6f
1         5  849766ff-6b13-4de9-8169-e3cfb6832bbe
2         3  4f83ccd2-5dad-4420-89c2-8365a60846d4
3         3  1fcd017d-8cdb-4901-b874-39e43b3bbb0b
4         3  a088b206-5765-4a0d-b384-1ed9dc274246
1000


In [337]:
#Rearrange the sequence of the dataframe
stores_orders_table_df = stores_orders_table_df[['order_id', 'store_id']]

print(stores_orders_table_df.head())

                               order_id  store_id
0  140ead02-1500-4660-897e-8773e7c34a6f         5
1  849766ff-6b13-4de9-8169-e3cfb6832bbe         5
2  4f83ccd2-5dad-4420-89c2-8365a60846d4         3
3  1fcd017d-8cdb-4901-b874-39e43b3bbb0b         3
4  a088b206-5765-4a0d-b384-1ed9dc274246         3


In [476]:
#Direct insert using the dataframe generated: 
#There should be 1000 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
stores_orders_insert = [tuple(row) for row in stores_orders_table_df.values]

for row in stores_orders_insert:
    cur.execute("INSERT INTO stores_orders (order_id, store_id) VALUES (%s, %s);", row)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#stores_orders_table_df = pd.read_csv("storesorders Table Insertion.csv")

#Rearrange the sequence of the dataframe
#stores_orders_table_df = stores_orders_table_df[['order_id', 'store_id']]

#stores_orders_insert = [tuple(row) for row in stores_orders_table_df.values]

#for row in stores_orders_insert:
    #cur.execute("INSERT INTO stores_orders (order_id, store_id) VALUES (%s, %s);", row)

#conn.commit()

In [477]:
#check the table and the values
cur.execute("SELECT * FROM stores_orders;")
rows_storessales = cur.fetchall()
for value in rows_storessales:
    print(value)

print(len(rows_storessales))

('140ead02-1500-4660-897e-8773e7c34a6f', 5)
('849766ff-6b13-4de9-8169-e3cfb6832bbe', 5)
('4f83ccd2-5dad-4420-89c2-8365a60846d4', 3)
('1fcd017d-8cdb-4901-b874-39e43b3bbb0b', 3)
('a088b206-5765-4a0d-b384-1ed9dc274246', 3)
('0cbbbc5b-b146-4449-83e2-f7e3b2adcbd7', 5)
('cd963182-f3de-464d-87a1-6cbaa08a6be5', 4)
('f6c6e797-4799-4dda-aca1-1e39377a6ba2', 1)
('85911ba6-8f8d-4336-bbbc-b9ff3181d608', 4)
('53c95ac8-d16a-46b1-a66b-a0d52c4c34ad', 1)
('76819cd7-29f5-4432-9353-4db841bd3b4a', 3)
('7d643604-9f8a-45a0-935f-1a2f4bd49a7d', 1)
('97fcf51d-e661-4847-9e8b-c56c18d1e28d', 2)
('f5050e82-9242-4e06-8e66-182429448018', 2)
('02296da9-4abf-4be5-af59-0e44e5a4ad82', 1)
('c784b6b6-a242-467b-98ca-3b87aede7441', 5)
('1bf8278a-7398-42b8-9dbf-d41580a9c250', 1)
('dd76b9cd-6928-4ded-b590-dd929fedb155', 2)
('e51dd881-85af-4224-8b41-82b828a9a729', 3)
('42a4de1f-e58a-439d-93cb-caddbae19482', 5)
('798bdca3-37e7-46f3-8c44-97142715c478', 4)
('f45be451-38c3-48a2-aa26-907a3c4a266b', 4)
('605542e6-6b5f-4c13-bfff-8cfde2

### Insert Data to product_orders Table
Two method available:
- direct insert using the dataframe generated: product_orders_table_df
- read in the csv file generated from previous step

In [338]:
print(product_orders_table_df.head())
print(len(product_orders_table_df))

   product_id                              order_id  quantity expiration_date  \
0           1  140ead02-1500-4660-897e-8773e7c34a6f        18      2024-08-21   
1           2  140ead02-1500-4660-897e-8773e7c34a6f       100      2025-02-27   
2           3  140ead02-1500-4660-897e-8773e7c34a6f        93      2024-11-24   
3           4  140ead02-1500-4660-897e-8773e7c34a6f        94      2025-05-22   
4           5  140ead02-1500-4660-897e-8773e7c34a6f        76      2024-08-05   

   unit_price  
0        3.04  
1       42.15  
2       72.48  
3       82.37  
4       86.02  
4999


In [340]:
#Rearrange the columns to fit the format
product_orders_table_df = product_orders_table_df[['product_id', 'order_id', 'expiration_date', 'quantity', 'unit_price']]

#Convert expiration_date to datetime format
product_orders_table_df['expiration_date'] = pd.to_datetime(product_orders_table_df['expiration_date'])

print(product_orders_table_df.head())

   product_id                              order_id expiration_date  quantity  \
0           1  140ead02-1500-4660-897e-8773e7c34a6f      2024-08-21        18   
1           2  140ead02-1500-4660-897e-8773e7c34a6f      2025-02-27       100   
2           3  140ead02-1500-4660-897e-8773e7c34a6f      2024-11-24        93   
3           4  140ead02-1500-4660-897e-8773e7c34a6f      2025-05-22        94   
4           5  140ead02-1500-4660-897e-8773e7c34a6f      2024-08-05        76   

   unit_price  
0        3.04  
1       42.15  
2       72.48  
3       82.37  
4       86.02  


In [478]:
#Direct insert using the dataframe generated: 
#There should be 4999 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
product_orders_insert = [tuple(row) for row in product_orders_table_df.values]

for data in product_orders_insert:
    cur.execute("INSERT INTO product_orders (product_id, order_id, expiration_date, quantity, unit_price) VALUES (%s, %s, %s, %s, %s);", data)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#product_orders_table_df = pd.read_csv("productorders Table Insertion.csv")

#Rearrange the columns to fit the format
#product_orders_table_df = product_orders_table_df[['product_id', 'order_id', 'expiration_date', 'quantity', 'unit_price']]

#Convert expiration_date to datetime format
#product_orders_table_df['expiration_date'] = pd.to_datetime(product_orders_table_df['expiration_date'])

#product_orders_insert = [tuple(row) for row in product_orders_table_df.values]

#for data in product_orders_insert:
    #cur.execute("INSERT INTO product_orders (product_id, order_id, expiration_date, quantity, unit_price) VALUES (%s, %s, %s, %s, %s);", data)

#conn.commit()

In [479]:
#check the table and the values
cur.execute("SELECT * FROM product_orders;")
rows_productorders = cur.fetchall()
for value in rows_productorders:
    print(value)

print(len(rows_productorders))

(1, '140ead02-1500-4660-897e-8773e7c34a6f', datetime.date(2024, 8, 21), 18, Decimal('3.04'))
(2, '140ead02-1500-4660-897e-8773e7c34a6f', datetime.date(2025, 2, 27), 100, Decimal('42.15'))
(3, '140ead02-1500-4660-897e-8773e7c34a6f', datetime.date(2024, 11, 24), 93, Decimal('72.48'))
(4, '140ead02-1500-4660-897e-8773e7c34a6f', datetime.date(2025, 5, 22), 94, Decimal('82.37'))
(5, '140ead02-1500-4660-897e-8773e7c34a6f', datetime.date(2024, 8, 5), 76, Decimal('86.02'))
(4, '849766ff-6b13-4de9-8169-e3cfb6832bbe', datetime.date(2025, 5, 20), 5, Decimal('3.74'))
(6, '849766ff-6b13-4de9-8169-e3cfb6832bbe', datetime.date(2025, 11, 13), 58, Decimal('93.59'))
(7, '849766ff-6b13-4de9-8169-e3cfb6832bbe', datetime.date(2026, 1, 6), 95, Decimal('2.50'))
(8, '849766ff-6b13-4de9-8169-e3cfb6832bbe', datetime.date(2025, 12, 13), 45, Decimal('49.83'))
(9, '849766ff-6b13-4de9-8169-e3cfb6832bbe', datetime.date(2025, 11, 12), 45, Decimal('70.27'))
(10, '4f83ccd2-5dad-4420-89c2-8365a60846d4', datetime.date(20

### Insert Data to SpecificSales Table
Two method available:
- direct insert using the dataframe generated: specific_sales_final
- read in the csv file generated from previous step

In [341]:
print(specific_sales_final.head())
print(len(specific_sales_final))

   sales_id  product_id  quantity  price_paid
0         1           2         8      424.64
1         1           5         8      670.64
2         1          10         7      428.12
3         1          12         1      117.04
4         1          14        10     1476.20
18333


In [480]:
#Direct insert using the dataframe generated: 
#There should be 18333 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
specific_sales_insert = [tuple(row) for row in specific_sales_final.values]

for sales_data in specific_sales_insert:
    cur.execute("INSERT INTO SpecificSales (sales_id, product_id, quantity, price_paid) VALUES (%s, %s, %s, %s);", sales_data)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#specific_sales_final = pd.read_csv("SpecificSales Table Insertion.csv")

#specific_sales_insert = [tuple(row) for row in specific_sales_final.values]

#for sales_data in specific_sales_insert:
    #cur.execute("INSERT INTO SpecificSales (sales_id, product_id, quantity, price_paid) VALUES (%s, %s, %s, %s);", sales_data)

#conn.commit()

In [481]:
#check the table and the values
cur.execute("SELECT * FROM SpecificSales;")
rows_specificsales = cur.fetchall()
for value in rows_specificsales:
    print(value)

print(len(rows_specificsales))

(1, 2, 8, Decimal('424.64'))
(1, 5, 8, Decimal('670.64'))
(1, 10, 7, Decimal('428.12'))
(1, 12, 1, Decimal('117.04'))
(1, 14, 10, Decimal('1476.20'))
(1, 20, 10, Decimal('261.30'))
(1, 21, 8, Decimal('541.44'))
(1, 22, 10, Decimal('468.60'))
(1, 24, 6, Decimal('345.36'))
(1, 25, 6, Decimal('934.92'))
(1, 27, 1, Decimal('154.62'))
(1, 31, 8, Decimal('541.22'))
(2, 2, 10, Decimal('655.30'))
(2, 5, 2, Decimal('13.32'))
(2, 7, 12, Decimal('919.37'))
(2, 9, 6, Decimal('297.96'))
(2, 14, 9, Decimal('971.46'))
(2, 27, 10, Decimal('22.50'))
(2, 28, 10, Decimal('1027.46'))
(2, 29, 10, Decimal('25.10'))
(2, 30, 6, Decimal('480.96'))
(3, 5, 8, Decimal('543.12'))
(3, 9, 8, Decimal('10.64'))
(3, 11, 2, Decimal('27.48'))
(3, 14, 8, Decimal('545.04'))
(3, 20, 10, Decimal('1232.88'))
(3, 21, 10, Decimal('1172.60'))
(3, 22, 4, Decimal('193.00'))
(3, 25, 10, Decimal('1386.40'))
(3, 28, 4, Decimal('458.80'))
(3, 30, 2, Decimal('135.78'))
(3, 31, 5, Decimal('661.40'))
(3, 32, 7, Decimal('1032.29'))
(4, 1,

### Insert Data to Payment Table
Two method available:
- direct insert using the dataframe generated: payment_table
- read in the csv file generated from previous step

In [343]:
print(payment_table.head())
print(len(payment_table))

   sales_id payment_method  payment_id  discount  tax_amount  total_payment
0         1          Check           1        19      556.86        5711.78
1         2  Bank Transfer           2        11      386.18        4314.13
2         3           Cash           3        34      647.45        5531.07
3         4         PayPal           4        21      486.64        4880.35
4         5           Cash           5        16      236.76        2509.68
2000


In [344]:
#Rearrange the sequence for the dataframe for data insertion
desired_columns = ['payment_id', 'sales_id', 'payment_method', 'discount', 'tax_amount', 'total_payment']
#Reindex the DataFrame to match the desired sequence
payment_table_rearrange = payment_table.reindex(columns=desired_columns)

print(payment_table_rearrange.head())

   payment_id  sales_id payment_method  discount  tax_amount  total_payment
0           1         1          Check        19      556.86        5711.78
1           2         2  Bank Transfer        11      386.18        4314.13
2           3         3           Cash        34      647.45        5531.07
3           4         4         PayPal        21      486.64        4880.35
4           5         5           Cash        16      236.76        2509.68


In [482]:
#Direct insert using the dataframe generated: 
#There should be 2000 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
payment_insert = [tuple(row) for row in payment_table_rearrange.values]

for payment in payment_insert:
    cur.execute("INSERT INTO Payment (payment_id, sales_id, payment_method, discount, tax_amount, total_payment) VALUES (%s, %s, %s, %s, %s, %s);", payment)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#payment_table = pd.read_csv("Payment Table Insertion.csv")

#Rearrange the sequence for the dataframe for data insertion
#desired_columns = ['payment_id', 'sales_id', 'payment_method', 'discount', 'tax_amount', 'total_payment']
#Reindex the DataFrame to match the desired sequence
#payment_table_rearrange = payment_table.reindex(columns=desired_columns)

#payment_insert = [tuple(row) for row in payment_table_rearrange.values]

#for payment in payment_insert:
    #cur.execute("INSERT INTO Payment (payment_id, sales_id, payment_method, discount, tax_amount, total_payment) VALUES (%s, %s, %s, %s, %s, %s);", payment)

#conn.commit()

In [483]:
#check the table and the values
cur.execute("SELECT * FROM Payment;")
rows_payment = cur.fetchall()
for value in rows_payment:
    print(value)

print(len(rows_payment))

(1, 1, 'Check', 19, Decimal('556.86'), Decimal('5711.78'))
(2, 2, 'Bank Transfer', 11, Decimal('386.18'), Decimal('4314.13'))
(3, 3, 'Cash', 34, Decimal('647.45'), Decimal('5531.07'))
(4, 4, 'PayPal', 21, Decimal('486.64'), Decimal('4880.35'))
(5, 5, 'Cash', 16, Decimal('236.76'), Decimal('2509.68'))
(6, 6, 'Venmo', 1, Decimal('497.52'), Decimal('6126.66'))
(7, 7, 'PayPal', 6, Decimal('233.71'), Decimal('2744.46'))
(8, 8, 'Debit Card', 18, Decimal('802.80'), Decimal('8326.18'))
(9, 9, 'PayPal', 27, Decimal('442.87'), Decimal('4137.69'))
(10, 10, 'Bank Transfer', 14, Decimal('954.34'), Decimal('10334.18'))
(11, 11, 'Check', 40, Decimal('96.73'), Decimal('760.00'))
(12, 12, 'Credit Card', 7, Decimal('217.20'), Decimal('2525.77'))
(13, 13, 'PayPal', 18, Decimal('588.55'), Decimal('6104.14'))
(14, 14, 'Bank Transfer', 17, Decimal('52.00'), Decimal('545.31'))
(15, 15, 'Check', 23, Decimal('762.97'), Decimal('7477.13'))
(16, 16, 'Check', 3, Decimal('533.96'), Decimal('6453.32'))
(17, 17, 'Ca

### Insert Data to CustomerRating Table
Two method available:
- direct insert using the dataframe generated: complete_review
- read in the csv file generated from previous step

In [346]:
print(complete_review.head())
print(len(complete_review))

    customer_id  store_id         review_date  rating  \
0             1         3 2023-08-04 00:17:00     5.0   
13            2         2 2023-08-27 11:28:00     3.0   
25            3         1 2023-05-11 00:50:00     3.0   
38            4         3 2023-06-06 12:29:00     1.0   
47            5         4 2023-11-14 10:40:00     5.0   

                                            review  
0                 great selection of fresh produce  
13  a well-stocked supermarket with friendly staff  
25                      good for one-stop shopping  
38                                  not very clean  
47                great selection of fresh produce  
133


In [347]:
#Check for missing variable
missing_values_review = complete_review.isnull().sum()

print("Number of missing values:")
print(missing_values_review)

Number of missing values:
customer_id    0
store_id       0
review_date    0
rating         0
review         0
dtype: int64


In [350]:
#check the range of the rating and convert it to integer
# Check the range and data type of the 'rating' column
print(complete_review['rating'].describe())

# Check the data type of the 'rating' column
print(complete_review['rating'].dtype)

count    133.000000
mean       4.015038
std        1.193321
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64
int32


In [351]:
#Convert the 'rating' column to integers
complete_review['rating'] = complete_review['rating'].astype(int)

print(complete_review.head())


    customer_id  store_id         review_date  rating  \
0             1         3 2023-08-04 00:17:00       5   
13            2         2 2023-08-27 11:28:00       3   
25            3         1 2023-05-11 00:50:00       3   
38            4         3 2023-06-06 12:29:00       1   
47            5         4 2023-11-14 10:40:00       5   

                                            review  
0                 great selection of fresh produce  
13  a well-stocked supermarket with friendly staff  
25                      good for one-stop shopping  
38                                  not very clean  
47                great selection of fresh produce  


In [352]:
#Check the highest and lowest values of 'rating'
highest_rating = complete_review['rating'].max()
lowest_rating = complete_review['rating'].min()

print("Highest rating:", highest_rating)
print("Lowest rating:", lowest_rating)

Highest rating: 5
Lowest rating: 1


In [484]:
#Direct insert using the dataframe generated: 
#There should be 133 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence
customer_rating_insert = [tuple(row) for row in complete_review.values]

for row in customer_rating_insert:
    cur.execute("INSERT INTO CustomerRating (customer_id, store_id, review_date, rating, review)VALUES (%s, %s, %s, %s, %s);", row)

conn.commit()

In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#complete_review = pd.read_csv("CustomerRating Table Insertion.csv")

#Convert the 'rating' column to integers
#complete_review['rating'] = complete_review['rating'].astype(int)

#customer_rating_insert = [tuple(row) for row in complete_review.values]

#for row in customer_rating_insert:
    #cur.execute("INSERT INTO CustomerRating (customer_id, store_id, review_date, rating, review)VALUES (%s, %s, %s, %s, %s);", row)

#conn.commit()

In [485]:
#check the table and the values
cur.execute("SELECT * FROM CustomerRating;")
rows_review = cur.fetchall()
for value in rows_review:
    print(value)

print(len(rows_review))

(1, 3, datetime.datetime(2023, 8, 4, 0, 17), 5, 'great selection of fresh produce')
(2, 2, datetime.datetime(2023, 8, 27, 11, 28), 3, 'a well-stocked supermarket with friendly staff')
(3, 1, datetime.datetime(2023, 5, 11, 0, 50), 3, 'good for one-stop shopping')
(4, 3, datetime.datetime(2023, 6, 6, 12, 29), 1, 'not very clean')
(5, 4, datetime.datetime(2023, 11, 14, 10, 40), 5, 'great selection of fresh produce')
(6, 5, datetime.datetime(2024, 1, 27, 13, 27), 3, 'average prices, good selection')
(7, 3, datetime.datetime(2023, 5, 1, 15, 27), 4, 'a great place to buy groceries')
(8, 4, datetime.datetime(2024, 4, 27, 20, 22), 5, 'customer service is excellent')
(9, 1, datetime.datetime(2024, 3, 14, 8, 37), 3, 'low prices, but poor customer service')
(10, 3, datetime.datetime(2023, 5, 29, 13, 10), 4, 'friendly staff and good prices')
(11, 1, datetime.datetime(2023, 9, 30, 8, 14), 3, 'good selection of fresh produce')
(12, 3, datetime.datetime(2023, 10, 23, 14, 0), 5, 'good selection')
(13,

### Insert Data to OperatingCost Table
Two method available:
- direct insert using the dataframe generated: operatingcost_df
- read in the csv file generated from previous step

In [353]:
print(operatingcost_df.head())
print(len(operatingcost_df))

         date  store_id                               cost_description  \
0  2021-01-01         1  January rent payment, gas, water, electricity   
1  2021-01-01         2  January rent payment, gas, water, electricity   
2  2021-01-01         3  January rent payment, gas, water, electricity   
3  2021-01-01         4  January rent payment, gas, water, electricity   
4  2021-01-01         5  January rent payment, gas, water, electricity   

   variable_cost  fixed_cost  
0           1628        5000  
1           1875        9000  
2           1353        8250  
3           1163       14500  
4           1159       10000  
200


In [354]:
# Rearrange the columns
operatingcost_df = operatingcost_df.reindex(columns=['store_id', 'date', 'fixed_cost', 'variable_cost', 'cost_description'])

print(operatingcost_df.head())

   store_id        date  fixed_cost  variable_cost  \
0         1  2021-01-01        5000           1628   
1         2  2021-01-01        9000           1875   
2         3  2021-01-01        8250           1353   
3         4  2021-01-01       14500           1163   
4         5  2021-01-01       10000           1159   

                                cost_description  
0  January rent payment, gas, water, electricity  
1  January rent payment, gas, water, electricity  
2  January rent payment, gas, water, electricity  
3  January rent payment, gas, water, electricity  
4  January rent payment, gas, water, electricity  


In [355]:
#Make sure the date is in the right format
operatingcost_df['date'] = pd.to_datetime(operatingcost_df['date'])

print(operatingcost_df.head())


   store_id       date  fixed_cost  variable_cost  \
0         1 2021-01-01        5000           1628   
1         2 2021-01-01        9000           1875   
2         3 2021-01-01        8250           1353   
3         4 2021-01-01       14500           1163   
4         5 2021-01-01       10000           1159   

                                cost_description  
0  January rent payment, gas, water, electricity  
1  January rent payment, gas, water, electricity  
2  January rent payment, gas, water, electricity  
3  January rent payment, gas, water, electricity  
4  January rent payment, gas, water, electricity  


In [486]:
#Direct insert using the dataframe generated: 
#There should be 200 rows in the SQL database
#The order of columns in the dataframe is already in the right sequence

operatingcost_insert = [tuple(row) for row in operatingcost_df.values]

for cost in operatingcost_insert:
    cur.execute("INSERT INTO OperatingCost (store_id, date, fixed_cost, variable_cost, cost_description) VALUES (%s, %s, %s, %s, %s);", cost)

conn.commit()


In [ ]:
#Indirect method by reading in the csv file created in previous step
#Remove # to us this method

#operatingcost_df = pd.read_csv("OperatingCost Table Insertion.csv")

# Rearrange the columns
#operatingcost_df = operatingcost_df.reindex(columns=['store_id', 'date', 'fixed_cost', 'variable_cost', 'cost_description'])

#Make sure the date is in the right format
#operatingcost_df['date'] = pd.to_datetime(operatingcost_df['date'])

#operatingcost_insert = [tuple(row) for row in operatingcost_df.values]

#for cost in operatingcost_insert:
    #cur.execute("INSERT INTO OperatingCost (store_id, date, fixed_cost, variable_cost, cost_description) VALUES (%s, %s, %s, %s, %s);", cost)

#conn.commit()



In [487]:
#check the table and the values
cur.execute("SELECT * FROM OperatingCost;")
rows_operatingcost = cur.fetchall()
for value in rows_operatingcost:
    print(value)

print(len(rows_operatingcost))

(1, datetime.date(2021, 1, 1), Decimal('5000.00'), Decimal('1628.00'), 'January rent payment, gas, water, electricity')
(2, datetime.date(2021, 1, 1), Decimal('9000.00'), Decimal('1875.00'), 'January rent payment, gas, water, electricity')
(3, datetime.date(2021, 1, 1), Decimal('8250.00'), Decimal('1353.00'), 'January rent payment, gas, water, electricity')
(4, datetime.date(2021, 1, 1), Decimal('14500.00'), Decimal('1163.00'), 'January rent payment, gas, water, electricity')
(5, datetime.date(2021, 1, 1), Decimal('10000.00'), Decimal('1159.00'), 'January rent payment, gas, water, electricity')
(1, datetime.date(2021, 2, 1), Decimal('5000.00'), Decimal('1673.00'), 'February rent payment, gas, water, electricity')
(2, datetime.date(2021, 2, 1), Decimal('9000.00'), Decimal('1442.00'), 'February rent payment, gas, water, electricity')
(3, datetime.date(2021, 2, 1), Decimal('8250.00'), Decimal('1541.00'), 'February rent payment, gas, water, electricity')
(4, datetime.date(2021, 2, 1), Deci

In [488]:
# Close the cursor and connection
cur.close()
conn.close()